In [72]:
import os
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind_from_stats
from scipy.stats import f_oneway as anova
import scipy
import plotly.graph_objs as go
from plotly.offline import plot
import json
import plotly.io as pio
from plotly.offline import iplot
import plotly as py
from matplotlib_venn import venn2, venn3
import matplotlib.pyplot as plt
import matplotlib
import plotly.colors
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import t
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
from pathlib import Path
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
  
import math
from select import select
import numpy as np
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

#only used for the app

# import django
# from django.conf import settings
# from django.contrib.auth.decorators import login_required, permission_required
# from file_manager.models import DataAnalysisQueue, SampleRecord, \
#     SavedVisualization, VisualizationApp, UserSettings, ProcessingApp
# from django.shortcuts import render
# from django.conf import settings


In [73]:
#constants 
saved_settings ={}
plot_options = {}
JUPYTER_MODE = "JPY_PARENT_PID" in os.environ #check if it's in jupiter notebook mode
APPFOLDER = "./"
url_base = None

#settings
WRITE_OUTPUT = True
USE_MaxLFQ = False


In [74]:
''' This is only for the webapp, not for jupyter notebook
def get_run_name(queue_id):
    """_Get the run name/sample list from the result files, only
    used for webapp for populate dropdown list_
    Args:
        queue_id (_int_): _task id from the process queue_
    Returns:
        _type_: _pandas data serial contains experiment list_
        0              sample1
        1              sample2
        2              sample3
    """
    if not queue_id:
        return None
    # get processing name
    process_app = DataAnalysisQueue.objects.filter(
        pk=queue_id).first().processing_app.name
    # fragpipe results
    if "FragPipe" in process_app:
        peptide_file = DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_2
        peptide = pd.read_table(peptide_file)
        #
        # get experiment names from columns names containning " MaxLFQ Intensity"
        run_metadata = [
            col for col in peptide.columns if " MaxLFQ Intensity" in col]
        # remove " MaxLFQ Intensity" from the experiment names
        run_metadata = [name.replace(" MaxLFQ Intensity", "")
                            for name in run_metadata]
        # create a pandas series to store the experiment names
        run_metadata = pd.Series(run_metadata)
    elif "PD" in process_app:
        inpufile_6 = DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_6
        meta_table = pd.read_table(inpufile_6)
        # Replace single backslashes with forward slashes in the 'file_paths' column
        meta_table['File Name'] = meta_table['File Name'].str.replace('\\', '/', regex=False)
        # Apply a lambda function to extract file names without extensions
        meta_table['file_names'] = meta_table['File Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
        run_metadata =meta_table['file_names']

    else:
        run_metadata = pd.Series()

    return run_metadata
'''

' This is only for the webapp, not for jupyter notebook\ndef get_run_name(queue_id):\n    """_Get the run name/sample list from the result files, only\n    used for webapp for populate dropdown list_\n    Args:\n        queue_id (_int_): _task id from the process queue_\n    Returns:\n        _type_: _pandas data serial contains experiment list_\n        0              sample1\n        1              sample2\n        2              sample3\n    """\n    if not queue_id:\n        return None\n    # get processing name\n    process_app = DataAnalysisQueue.objects.filter(\n        pk=queue_id).first().processing_app.name\n    # fragpipe results\n    if "FragPipe" in process_app:\n        peptide_file = DataAnalysisQueue.objects.filter(\n            pk=queue_id).first().output_file_2\n        peptide = pd.read_table(peptide_file)\n        #\n        # get experiment names from columns names containning " MaxLFQ Intensity"\n        run_metadata = [\n            col for col in peptide.column

In [75]:
#This is only for the webapp, not for jupyter notebook

# def queue_info_api(queue_id, server_address, user_name, password):
#     """_Get the queue and app info from the server through API, this
#     is only for the jupyter notebook, not for the webapp_
#     """

#     authinfo = HTTPBasicAuth(user_name, password)

#     #get queue info and test if the user name and password are correct   

#     queue_response = requests.get(
#         f'http://{server_address}/files/api/DataAnalysisQueue/{queue_id}/',
#         auth=authinfo
#     )
#     if queue_response.status_code != 200:
#         raise Exception("Invalid username or password")
#     else:
#         queue_json_data = queue_response.json()

#     # Get app information
#     app_response = requests.get(
#         f"http://{server_address}/files/api/ProcessingApp/{queue_json_data['processing_app']}/",
#         auth=authinfo
#     )
#     app_json_data = app_response.json()
#     return queue_json_data, app_json_data


In [76]:
def sumIDs(IDMatrix):
    """_summarize the ID matrix infor into ID summary_
    

    Args:
        IDMatrix (_type_): _protein or pepetides matrix_
        0 Accession/Annotated Sequence 	run1 	run2 	run3 
        1 P023D12	MS2 	MBR 	NaN 
        2 P1222	NaN 	ID 	NaN 
    ID: means we don't know the ID mode

    Returns:
        _type_: _description_
                                      names  MS2_IDs  MBR_IDs  Total_IDs
0            10ng_QC_1_channel2 Intensity      NaN      NaN       3650
1            10ng_QC_2_channel1 Intensity      NaN      NaN       3604
....
    """
    # removes the columns that don't have ID data
    columns = [col for col in IDMatrix.columns if not any(
        substring in col for substring in [
            'Accession', 'Annotated Sequence'])]
    #put each ID_Modes into a list
    returnNames = []
    MS2_ID = []
    MBR_ID = []
    total_ID = []
    for eachColumn in columns:
        MS2_ID.append(len(IDMatrix[eachColumn][IDMatrix[eachColumn] == "MS2"])) #PD differentiates
        MBR_ID.append(len(IDMatrix[eachColumn][IDMatrix[eachColumn] == "MBR"])) #PD differentiates
        total_ID_each = len(IDMatrix[eachColumn][IDMatrix[eachColumn] == "ID"]) #some don't so we count total directly
        if total_ID_each == 0: #otherwise we sum
            total_ID_each = len(IDMatrix[eachColumn][
                IDMatrix[eachColumn] == "MS2"]) + len(IDMatrix[
                eachColumn][IDMatrix[eachColumn] == "MBR"])
        total_ID.append(total_ID_each)

    return pd.DataFrame({'Run Identifier': columns,
                         'MS2_IDs': MS2_ID,
                         'MBR_IDs': MBR_ID,
                         'Total_IDs': total_ID})



In [77]:
def generate_column_from_name_mapping(columns, partial_column_name_mapping):
    #input is column names, and a dictionary with what you want each column (key) to be renamed to (value)
    column_name_mapping = {}
    for col in columns:
        for key, value in partial_column_name_mapping.items():
            if key in col: #in the case of PD, we are looking for a pattern within the column name
                column_name_mapping[col] = value
                break
    return column_name_mapping

In [78]:
def generate_column_to_name_mapping(columns, partial_column_name_mapping):
    #input is column names, and a dictionary with what you want each column (key) to be renamed to (value)
    column_name_mapping = {}
    for col in columns:
        for key, value in partial_column_name_mapping.items():
            if key == col:  #after we get away from PD's weirdness, then we want exact matches,
                            #so we don't get for example 1-11 when we look for 1-1 for file Identifiers or filenames
                column_name_mapping[col] = value
                break
    return column_name_mapping

In [79]:
def combine_IDs(all_matrix, MS2_matrix):
    # make IDs into MBR
    if "Annotated Sequence" in all_matrix.columns:
        name = "Annotated Sequence"
    elif "Accession" in all_matrix.columns:  
        name = "Accession"
    id_cols = all_matrix.columns.tolist()
    id_cols.remove(name)
    
    
    # for eachColumn in id_cols:
    #     if len(all_matrix[(all_matrix[name].isin(MS2_matrix[name]) & (MS2_matrix[eachColumn] == "MS2"))]) > 0:
    #         all_matrix.loc[(all_matrix[name].isin(MS2_matrix[name]) & (MS2_matrix[eachColumn] == "MS2")), [eachColumn]] = MS2_matrix[[eachColumn]]

    all_keys = pd.merge(all_matrix[name], MS2_matrix[name],how="outer")
    # print(all_keys)

    all_matrix = pd.merge(all_matrix, all_keys, how="right").replace("ID","MBR")
    MS2_matrix = pd.merge(MS2_matrix, all_keys, how="right")

    # print(all_matrix)
    for eachColumn in id_cols:
        # print(len(all_matrix[eachColumn]))
        # print(len(MS2_matrix[eachColumn]))
        if len(all_matrix[(all_matrix[name].isin(MS2_matrix[name]) & (MS2_matrix[eachColumn] == "MS2"))]) > 0:
            all_matrix.loc[(all_matrix[name].isin(MS2_matrix[name]) & (MS2_matrix[eachColumn] == "MS2")), [eachColumn]] = "MS2"

    # print(all_matrix)

    return all_matrix #noticed this changed

In [80]:
def read_file(queue_id=None, queue_info= None, processor_info = None,
               input1=None, input2=None,input3=None, input4=None, input5=None,
            process_app = None, file_id = 1, filter_out_keratin = True):
    """_Read data from data manager API or through local files or read directly
    in the webapp_
    Args:
        queue_id (_int_): _processing queue id_
        queue_info (_dict_): _queue info from the API_
        processor_info (_dict_): _processor info from the API_        
        input1 (_str_): _input file 1_ 
        input2 (_str_): _input file 2_
        input3 (_str_): _input file 3_
        input4 (_str_): _input file 4_
        input5 (_str_): _input file 5_
        process_app (_str_): _process app name_
    Returns:
        _dict_: _dictionary containing data all data        
    """

    """ Input files are as followws
    App       Input file
              1                         2                       3         4         5
    PD        _Proteins                 _PeptideGroups                              _InputFiles
    Fragpipe  combined_protein          combined_peptide
    DIANN     diann-output.pg_matrix    diann-output.pr_matrix  protein   peptide   filelist_diann.txt
    """

    min_unique_peptides = 1

    #getting files from data system
        # getting files from data system (webapp)
    if queue_id is not None and processor_info is None:
        # Method 1 pull data directly (used by the webapp)
        process_app = DataAnalysisQueue.objects.filter(
            pk=queue_id).first().processing_app.name
        input1= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_1
        input2= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_2  
        input3= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_3
        input4= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_4  
        input5= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_5
    elif queue_info is not None and processor_info is not None:
    # Method 2 pull data from the data system (used by jupyter notebook)
        process_app = processor_info["name"]
        input1= queue_info["output_file_1"]
        input2= queue_info["output_file_2"]  
        input3= queue_info["output_file_3"]
        input4= queue_info["output_file_4"]  
        input5= queue_info["output_file_5"]
    # method 3 feed data directly (through local file paths)
    else:
        analysis_file = input1

    if "FragPipe" in process_app:     # fragpipe results
        # read data
        peptide_table = pd.read_table(input2,low_memory=False)
        protein_table = pd.read_table(input1,low_memory=False)

        # filter Contaminant
        peptide_table= peptide_table[~peptide_table[
            'Mapped Proteins'].str.contains(
            "contam_sp", na=False)]
        protein_table= protein_table[~protein_table['Protein'].str.contains(
            "contam_sp", na=False)].query(
            "`Combined Total Peptides` >= @min_unique_peptides")
        # get experiment names from columns names containning "Intensity"
        # or " MaxLFQ Intensity" if MaxLFQ is used

        if USE_MaxLFQ:
            column_tail = " MaxLFQ Intensity"
            intensity_cols = protein_table.columns[
                protein_table.columns.str.contains(column_tail)].tolist() #These should be the same for MS2
        else:
            column_tail = " Intensity" #this includes MaxLFQ Intensity
            cols = protein_table.columns
            intensity_cols =  protein_table.columns[( \
                cols.str.contains(" Intensity")) & \
                (~cols.str.contains(" MaxLFQ Intensity"))]
        prot_ID_cols = protein_table.columns[(cols.str.contains(" Unique Spectral Count")) & ~(cols == "Combined Unique Spectral Count")] #so we remove MaxLFQ Intensity here
        cols = peptide_table.columns
        pep_ID_cols = peptide_table.columns[(cols.str.contains(" Spectral Count"))] #so we remove MaxLFQ Intensity here
        #get the column names of protein_table that contain "Intensity" but not " MaxLFQ"
        
        # ALL
        ## Proteins abundance table
        protein_table.rename(columns={'Protein ID': 'Accession'},inplace=True)
        all_abundance_cols = intensity_cols.append(pd.Index(['Accession']))
        prot_abundance = protein_table.loc[:, all_abundance_cols]
        ## Peptide abundance table
        peptide_table.rename(columns={'Peptide Sequence': 'Annotated Sequence'}, inplace=True)
        all_abundance_cols = intensity_cols.append(pd.Index(['Annotated Sequence']))
        pep_abundance = peptide_table.loc[:, all_abundance_cols]

        # Proteins ID table
        all_ID_cols = prot_ID_cols.append(pd.Index(['Accession']))
        prot_ID_MS2 = protein_table.loc[:, all_ID_cols]
        
        # Proteins ID table
        all_ID_cols = pep_ID_cols.append(pd.Index(['Annotated Sequence']))
        pep_ID_MS2 = peptide_table.loc[:, all_ID_cols]


        # remove "Spectral Count", " MaxLFQ Intensity" or " Intensity" from names
        run_name_list = [name.replace(column_tail, "")
                            for name in intensity_cols]
        
        run_name_list = pd.DataFrame({"Run Names": run_name_list})
        run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: str(file_id) + "-" + str(x))

        prot_abundance = prot_abundance.rename(columns={
            col: col.replace(column_tail, "") for col in
              prot_abundance.columns if column_tail in col})

        pep_abundance = pep_abundance.rename(columns={
            col: col.replace(column_tail, "") for col in
              pep_abundance.columns if column_tail in col})
        
        prot_ID_MS2 = prot_ID_MS2.rename(columns={
            col: col.replace(" Unique Spectral Count", "") for col in
              prot_ID_MS2.columns if " Unique Spectral Count" in col})

        pep_ID_MS2 = pep_ID_MS2.rename(columns={
            col: col.replace(" Spectral Count", "") for col in
              pep_ID_MS2.columns if " Spectral Count" in col})

        # print(pep_ID_MS2.columns)

        for item in [prot_abundance,pep_abundance,pep_ID_MS2,prot_ID_MS2]:
            # Generate a new column name mapping using the function
            fileid_mapping = generate_column_to_name_mapping(item.columns, dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))
            item.rename(columns = fileid_mapping,inplace=True)
        

        # get ID matrix tables
        prot_ID = prot_abundance.copy()
        cols = [col for col in prot_ID.columns if col != 'Accession']
        for col in cols:
            if prot_ID[col].dtype != 'object': # Check if not a string column
                prot_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                prot_ID[col] = prot_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        pep_ID = pep_abundance.copy()
        cols = [col for col in pep_ID.columns if col != 'Annotated Sequence	']
        for col in cols:
            if pep_ID[col].dtype != 'object': # Check if not a string column
                pep_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                pep_ID[col] = pep_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        #Rename protein numbers
        
        cols = [col for col in prot_ID_MS2.columns if col != 'Accession']
        for col in cols:
            if prot_ID_MS2[col].dtype != 'object': # Check if not a string column
                prot_ID_MS2[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                prot_ID_MS2[col] = prot_ID_MS2[col].astype(str).str.replace("\d+\.\d+", "MS2", regex=True)
        cols = [col for col in pep_ID_MS2.columns if col != 'Annotated Sequence	']
        for col in cols:
            if pep_ID_MS2[col].dtype != 'object': # Check if not a string column
                pep_ID_MS2[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                pep_ID_MS2[col] = pep_ID_MS2[col].astype(str).str.replace("\d+\.\d+", "MS2", regex=True)

        # print(run_name_list)
        pep_ID = combine_IDs(pep_ID, pep_ID_MS2)
        prot_ID = combine_IDs(prot_ID, prot_ID_MS2)

        prot_other_info = protein_table.loc[
            :, ~protein_table.columns.str.contains('Intensity')]
        prot_other_info["Source_File"] = input1
        pep_other_info = peptide_table.loc[
            :, ~peptide_table.columns.str.contains('Intensity')]
        pep_other_info["Source_File"] = input2


    elif "DIANN" in process_app:
        # read in DIANN output files
        peptide_table = pd.read_table(input2,low_memory=False)
        protein_table = pd.read_table(input1,low_memory=False)
        protein_table_MS2 = pd.read_table(input3,low_memory=False)
        peptide_table_MS2 = pd.read_table(input4,low_memory=False)
        
        prot_other_info = pd.DataFrame({"Protein": protein_table["Protein.Ids"], "Protein.Group": protein_table["Protein.Group"]})
        pep_other_info = pd.DataFrame({"Mapped Proteins": peptide_table["Protein.Group"], "Modified.Sequence": peptide_table["Modified.Sequence"]})

        prot_other_info["Source_File"] = "None"
        pep_other_info["Source_File"] = "None"

        if filter_out_keratin:
            protein_table_MS2 = protein_table_MS2[~protein_table_MS2['First.Protein.Description'].str.contains(
            "Keratin", na=False)]
            protein_table = protein_table[~protein_table['First.Protein.Description'].str.contains(
            "Keratin", na=False)]
        # meta_table = pd.read_csv(input5, sep=' ', header=None, names=["File Name"])
        # filter Contaminant
        protein_table= protein_table[~protein_table['Protein.Group'].str.contains(
            "contam", na=False)]
        peptide_table= peptide_table[~peptide_table['Protein.Group'].str.contains(
            "contam", na=False)]
        protein_table_MS2= protein_table_MS2[~protein_table_MS2['Protein.Group'].str.contains(
            "contam", na=False)]
        peptide_table_MS2= peptide_table_MS2[~peptide_table_MS2['Protein.Group'].str.contains(
            "contam", na=False)]
        prot_other_info= prot_other_info[~prot_other_info['Protein'].str.contains(
            "contam", na=False)]
        pep_other_info= pep_other_info[~pep_other_info['Mapped Proteins'].str.contains(
            "contam", na=False)]
        
        
        prot_other_info.rename(columns={'Protein': 'Accession'}, inplace=True)
        pep_other_info.rename(columns={'Modified.Sequence': 'Annotated Sequence'}, inplace=True)
        # Replace backslashes with forward slashes if data comes from Windows
        # meta_table['File Name'] = meta_table['File Name'].str.replace('\\', '/', regex=False)
        # # Apply a lambda function to extract file names without extensions
        # meta_table['File Name'] = meta_table['File Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
        # run_name_list = meta_table['File Name'].tolist()
        # run_name_list = pd.DataFrame({"Run Names": run_name_list})
        # run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: str(file_id) + "-" + str(x))


        # Get the file names from the meta table
        protein_path_cols = protein_table_MS2.filter(regex='\\\\|Protein.Ids').columns

        ## Proteins
        prot_abundance = protein_table.loc[:, protein_path_cols]
        prot_abundance_MS2 = protein_table_MS2.loc[:, protein_path_cols]
        # Rename Columns to remove file path
        file_path_cols = protein_table.filter(regex='\\\\').columns
        prot_abundance.columns = [os.path.splitext(os.path.basename(x))[0] if x in file_path_cols else x for x in prot_abundance.columns]
        prot_abundance = prot_abundance.rename(columns={'Protein.Ids': 'Accession'})
        prot_abundance["Accession"] =  prot_abundance["Accession"].str.replace(";.*","",regex = True)
        prot_abundance_MS2.columns = [os.path.splitext(os.path.basename(x))[0] if x in file_path_cols else x for x in prot_abundance_MS2.columns]
        prot_abundance_MS2 = prot_abundance_MS2.rename(columns={'Protein.Ids': 'Accession'})   
        prot_abundance_MS2["Accession"] =  prot_abundance_MS2["Accession"].str.replace(";.*","",regex = True)

        ## Peptides
        peptide_path_cols = peptide_table_MS2.filter(regex='\\\\|Modified.Sequence').columns
        pep_abundance = peptide_table.loc[:, peptide_path_cols]
        pep_abundance_MS2 = peptide_table_MS2.loc[:, peptide_path_cols]
        # Rename Columns to remove file path
        file_path_cols = peptide_table.filter(regex='\\\\').columns
        pep_abundance.columns = [os.path.splitext(os.path.basename(x))[0] if x in file_path_cols else x for x in pep_abundance.columns]
        pep_abundance = pep_abundance.rename(columns={'Modified.Sequence': 'Annotated Sequence'})
        pep_abundance_MS2.columns = [os.path.splitext(os.path.basename(x))[0] if x in file_path_cols else x for x in pep_abundance_MS2.columns]
        pep_abundance_MS2 = pep_abundance_MS2.rename(columns={'Modified.Sequence': 'Annotated Sequence'})

        run_name_list = pd.DataFrame(data={"Run Names": [os.path.splitext(os.path.basename(x))[0] for x in file_path_cols]})
        run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: str(file_id) + "-" + str(x))

        for item in [prot_abundance,pep_abundance,prot_abundance_MS2,pep_abundance_MS2]:
            # Generate a new column name mapping using the function
            fileid_mapping = generate_column_to_name_mapping(item.columns, dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))
            item.rename(columns = fileid_mapping,inplace=True)


        #convert to str for IDs matrix
        pep_ID = pep_abundance.copy()
        cols = [col for col in pep_ID.columns if col != 'Accession']
        for col in cols:
            if pep_ID[col].dtype != 'object': # Check if not a string column
                pep_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                pep_ID[col] = pep_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        pep_ID_MS2 = pep_abundance_MS2.copy()
        cols = [col for col in pep_ID_MS2.columns if col != 'Accession']
        for col in cols:
            if pep_ID_MS2[col].dtype != 'object': # Check if not a string column
                pep_ID_MS2[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                pep_ID_MS2[col] = pep_ID_MS2[col].astype(str).str.replace("\d+\.\d+", "MS2", regex=True)
        prot_ID = prot_abundance.copy()
        cols = [col for col in prot_ID.columns if col != 'Accession']
        for col in cols:
            if prot_ID[col].dtype != 'object': # Check if not a string column
                prot_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                prot_ID[col] = prot_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        prot_ID_MS2 = prot_abundance_MS2.copy()
        cols = [col for col in prot_ID_MS2.columns if col != 'Accession']
        for col in cols:
            if prot_ID_MS2[col].dtype != 'object': # Check if not a string column
                prot_ID_MS2[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                prot_ID_MS2[col] = prot_ID_MS2[col].astype(str).str.replace("\d+\.\d+", "MS2", regex=True)

        pep_ID = combine_IDs(pep_ID, pep_ID_MS2)
        prot_ID = combine_IDs(prot_ID, prot_ID_MS2)       
        
    elif "PD" in process_app:
        peptide_table = pd.read_table(input2,low_memory=False)
        protein_table = pd.read_table(input1,low_memory=False)
        
        # filter Contaminant
        protein_table= protein_table[(protein_table[
            "Protein FDR Confidence: Combined"] == "High") &
                        ((protein_table["Master"] == "IsMasterProtein") | 
                         (protein_table["Master"] == "Master")) & 
                        (protein_table["Contaminant"] == False)]

        protein_table.rename(
            columns={'# Peptides': 'number of peptides'}, inplace=True)
        protein_table=protein_table.query(
            "`number of peptides` >= @min_unique_peptides")
        peptide_table= peptide_table[(peptide_table[
            'Contaminant'] == False) & (peptide_table["Confidence"]== "High")]

        meta_table = pd.read_table(input5,low_memory=False)
        #filter rows in meta table on File ID column if it is NaN
        meta_table = meta_table[meta_table['File ID'].notna()]

        # Replace single backslashes with forward slashes in the 'file_paths' column
        meta_table['File Name'] = meta_table['File Name'].str.replace('\\', '/', regex=False)
        # Apply a lambda function to extract file names without extensions
        meta_table['file_names'] = meta_table['File Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
        file_path_name_dict = dict(zip(meta_table['File ID'], meta_table['file_names']))
        run_name_list = pd.DataFrame({"Run Names": file_path_name_dict.values()})
        run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: str(file_id) + "-" + str(x))
        
        #format the read in table into three different tables: abundance, id and other_info
        prot_abundance = protein_table.filter(regex='Abundance:|Accession')
        prot_ID = protein_table.filter(regex='Found in Sample:|Accession')
        prot_other_info = protein_table.loc[:, ~protein_table.columns.str.contains('Found in Sample:|Abundance:')]
        

        pep_abundance = peptide_table.filter(regex='Abundance:|Annotated Sequence')
        pep_ID = peptide_table.filter(regex='Found in Sample:|Annotated Sequence')
        pep_other_info = peptide_table.loc[:, ~peptide_table.columns.str.contains('Found in Sample:|Abundance:')]

        prot_other_info["Source_File"] = input1
        pep_other_info["Source_File"] = input2

        #change column names to file/run names to our fileID

        new_dict = {"Abundance: " + key + ":": value for key, value in file_path_name_dict.items()}
        for item in [prot_abundance,pep_abundance]:
            # Generate a new column name mapping using the function
            column_name_mapping = generate_column_from_name_mapping(item.columns, new_dict)
            #TODO solving  A value is trying to be set on a copy of a slice from a DataFrame
            
            item.rename(columns = column_name_mapping, inplace = True)
            #use generate_column_to_name_mapping function because we don't want partial matches as in QC_HeLa.raw and QC_HeLa_20230727235101.raw
            fileid_mapping = generate_column_to_name_mapping(item.columns, dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))
            item.rename(columns = fileid_mapping,inplace=True)
        

        new_dict = {"Found in Sample: " + key + ":": value for key, value in file_path_name_dict.items()}
        for item in [pep_ID,prot_ID]:
            # Generate a new column name mapping using the function
            column_name_mapping = generate_column_from_name_mapping(item.columns, new_dict)

            item.rename(columns = column_name_mapping, inplace = True)
            #use generate_column_to_name_mapping function because we don't want partial matches
            fileid_mapping = generate_column_to_name_mapping(item.columns, dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))

            item.rename(columns = fileid_mapping,inplace=True)

        # replace "High" to MS2 "Peak Found" to MBR, the rest become np.NaN
        replacements = {'High': 'MS2', 'Peak Found': 'MBR', "Medium": np.NaN, "Low": np.NaN, "Not Found": np.NaN}
        for column in run_name_list["Run Identifier"]:
            if column in pep_ID.columns:
                pep_ID[column] = pep_ID[column].replace(to_replace=replacements)
    
            if column in prot_ID.columns:
                prot_ID[column] = prot_ID[column].replace(to_replace=replacements)
    
    # get ID summary by parsing ID Matrix
    protein_ID_summary = sumIDs(prot_ID)
    peptide_ID_summary = sumIDs(pep_ID)
    

    #sets the processing app in run_name_list
    run_name_list["Processing App"] = process_app
    run_name_list["Analysis Name"] = analysis_file


    return {'run_metadata': run_name_list,
            'protein_other_info': prot_other_info,
            'peptide_other_info': pep_other_info,
            'protein_abundance': prot_abundance,
            'protein_ID_matrix': prot_ID,
            'protein_ID_Summary': protein_ID_summary,
            'peptide_abundance': pep_abundance,
            'peptide_ID_matrix': pep_ID,
            'peptide_ID_Summary': peptide_ID_summary,

            }  



In [81]:
def read_files(queue_ids = None, queue_info = None, processor_info = None, grouped_input_files = []):
    '''
    Creates a list of data objects
    
    Input
    grouped_input_files
    [
        #File 0
    {input1:
    input2:
    input3:   
    input4:
    input5:
    process_app:
    },
        #File 1
    {input1:
    input2:
    input3:   
    input4:
    input5:
    process_app:
    },
    ...
    ]
    '''


    data_objects = []

    i = 0
    for eachGroup in grouped_input_files:
        if queue_ids is not None:
            pass
        else:
            process_app = eachGroup["process_app"]
            input1= eachGroup["input1"]
            input2= eachGroup["input2"]  
            input3= eachGroup["input3"]
            input4= eachGroup["input4"]  
            input5= eachGroup["input5"]

        current_data_object = read_file(input1=input1,input2=input2,
                                        input3=input3,input4 = input4,
                                        input5=input5, process_app=process_app,file_id = i)
        data_objects.append(current_data_object)
        
        i = i + 1

    
    return data_objects




In [82]:
def outer_join_data_objects(data_objects):
    '''
    Takes in a list of data objects as given by read_files and converts them to a single data object as given by read_files,
    protein info continues to show what was found on each original file, and so forth.
    '''

    first_file = True
    for eachDataObject in data_objects:
        print("***")
        if first_file:
            first_file = False
            final_data_object = eachDataObject
        else:
            final_data_object['run_metadata'] = pd.concat([final_data_object['run_metadata'],eachDataObject['run_metadata']]).reset_index(drop=True)
            final_data_object['protein_other_info'] = pd.concat([final_data_object['protein_other_info'],eachDataObject['protein_other_info']]).reset_index(drop=True)
            final_data_object['peptide_other_info'] = pd.concat([final_data_object['peptide_other_info'],eachDataObject['peptide_other_info']]).reset_index(drop=True)
            final_data_object['protein_ID_Summary'] = pd.concat([final_data_object['protein_ID_Summary'],eachDataObject['protein_ID_Summary']]).reset_index(drop=True)
            final_data_object['peptide_ID_Summary'] = pd.concat([final_data_object['peptide_ID_Summary'],eachDataObject['peptide_ID_Summary']]).reset_index(drop=True)
            duplicates_found = False
            
            #loop through to see if there are any duplicate files
            for eachCol in final_data_object['protein_abundance'].loc[:, final_data_object['protein_abundance'].columns!='Accession'].columns:
                if eachCol in eachDataObject['protein_abundance'].columns:
                    duplicates_found = True
                else:
                    pass
            for eachCol in final_data_object['protein_ID_matrix'].loc[:, final_data_object['protein_ID_matrix'].columns!='Accession'].columns:
                if eachCol in eachDataObject['protein_ID_matrix'].columns:
                    duplicates_found = True
                else:
                    pass
            for eachCol in final_data_object['peptide_abundance'].loc[:, final_data_object['peptide_abundance'].columns!='Annotated Sequence'].columns:
                if eachCol in eachDataObject['peptide_abundance'].columns:
                    duplicates_found = True
                else:
                    pass
            for eachCol in final_data_object['peptide_ID_matrix'].loc[:, final_data_object['peptide_ID_matrix'].columns!='Annotated Sequence'].columns:
                if eachCol in eachDataObject['peptide_ID_matrix'].columns:
                    duplicates_found = True
                else:
                    pass     
            if duplicates_found:
                print("Error: files analyzed twice present!!!")
                quit()
                print("@#afio2q3")
            else:
                #merge keeping all proteins
                # print("!!!!")
                final_data_object['protein_abundance'] = pd.merge(final_data_object['protein_abundance'],eachDataObject['protein_abundance'],how="outer")
                final_data_object['protein_ID_matrix'] = pd.merge(final_data_object['protein_ID_matrix'],eachDataObject['protein_ID_matrix'],how="outer")
                final_data_object['peptide_abundance'] = pd.merge(final_data_object['peptide_abundance'],eachDataObject['peptide_abundance'],how="outer")
                final_data_object['peptide_ID_matrix'] = pd.merge(final_data_object['peptide_ID_matrix'],eachDataObject['peptide_ID_matrix'],how="outer")
                
    return final_data_object

In [83]:
def filter_by_missing_values(data_object,
                             missing_value_thresh=33,
                             is_protein=True,
                             ignore_nan=False):
    """_Filter out proteins/peptides with missing values rate above the
    threshold_

    Args:
        data_object (_panada_): _dataframe contain data for one experimental
        condition_
        missing_value_thresh (int, optional): _description_. Defaults to 33.
        analysis_program (str, optional): _description_.
        ignore_nan: if filter intensity again with Nan threadshold, this 
        helps with the calcualting stdev step.

    Returns:
        _data_object_: _dictionary containing data for one experimental
         'abundances':        Accession  3_TrypsinLysConly_3A4_channel2 3_TrypsinLysConly_3BC_channel1
0     A0A096LP49                            0.00                                        10
1     A0A0B4J2D5                        89850.26                                      3311
2         A0AVT1                        83055.87                                    312312
    """
    if is_protein:
        name = "Accession"
        matrix_name = "protein_ID_matrix"
        other_info_name = "protein_other_info"
        abundance_name = "protein_abundance"
        
    else:
        name = "Annotated Sequence"
        matrix_name = "peptide_ID_matrix"
        other_info_name = "peptide_other_info"
        abundance_name = "peptide_abundance"

    #initializes number of missing values to zero
    protein_columns = data_object[matrix_name].assign(missingValues=0)

    i = 0
    # found all the proteins/peptides with missing values rate below
    # the threshold, pep_columns contains the remaining protein/peptide
    # in a pandas dataframe with $names as its column name
    for each_column in data_object[matrix_name].loc[
            :, ~data_object[matrix_name].columns.str.contains(
                name)].columns:
        # replace "nan" to np.nan
        protein_columns = protein_columns.replace({"nan": np.nan}) 
        #find missing values and increment those rows (a row is a protein/peptide) total number of missing values

        protein_columns.loc[(protein_columns[each_column] != "MS2")
                            &(protein_columns[each_column] != "MBR")
                            &(protein_columns[each_column] != "ID"), #this is more robust than using nan's in case something fails to convert
                             "missingValues"] += 1

        i += 1

    protein_columns = protein_columns.assign(missingValuesRate=(
        protein_columns["missingValues"] / i) * 100)
    
    protein_columns = protein_columns.query(
        "missingValuesRate < @missing_value_thresh")
    
    protein_columns = protein_columns.loc[:,
                                  protein_columns.columns.str.contains(name)]

    # filter the data_object with the remaining proteins/peptides names
    data_object[abundance_name] = protein_columns.merge(
        data_object[abundance_name])
    data_object[matrix_name] = protein_columns.merge(
        data_object[matrix_name])
    data_object[other_info_name] = protein_columns.merge(
        data_object[other_info_name])
    # In case there is mismatch between ID table and abundance table,
    # mannually remove the row with all NaN values
    # keep rows in data_object[abundance_name] where at least two values are 
    # not NaN(do this to all rows except the first row), otherwise can't
    # calculate the stdev
    if ignore_nan:
        data_object[abundance_name] = data_object[abundance_name].dropna(
            thresh=2, subset=data_object[abundance_name].columns[1:])
        # This will cause the veen diagram to be different from R program
    
    return data_object

In [84]:
def filter_by_missing_values_MS2(data_object,
                             missing_value_thresh=33,
                             is_protein=True,
                             ignore_nan=False):
    """_Filter out proteins/peptides with missing values rate above the
    threshold_

    Args:
        data_object (_panada_): _dataframe contain data for one experimental
        condition_
        missing_value_thresh (int, optional): _description_. Defaults to 33.
        analysis_program (str, optional): _description_.
        ignore_nan: if filter intensity again with Nan threadshold, this 
        helps with the calcualting stdev step.

    Returns:
        _data_object_: _dictionary containing data for one experimental
         'abundances':        Accession  3_TrypsinLysConly_3A4_channel2
0     A0A096LP49                            0.00
1     A0A0B4J2D5                        89850.26
2         A0AVT1                        83055.87
    """
    if is_protein:
        name = "Accession"
        matrix_name = "protein_ID_matrix"
        other_info_name = "protein_other_info"
        abundance_name = "protein_abundance"
        
    else:
        name = "Annotated Sequence"
        matrix_name = "peptide_ID_matrix"
        other_info_name = "peptide_other_info"
        abundance_name = "peptide_abundance"

    protein_columns = data_object[matrix_name].assign(missingValues=0)

    i = 0
    # found all the proteins/peptides with missing values rate below
    # the threshold, pep_columns contains the remaining protein/peptide
    # in a pandas dataframe with $names as its column name
    for each_column in data_object[matrix_name].loc[:, ~data_object[matrix_name].columns.str.contains(name)].columns:
        # replace "nan" to np.nan
        protein_columns = protein_columns.replace({"nan": np.nan}) 
        protein_columns.loc[protein_columns[each_column] != "MS2", #ID/MBR are still missing values if you are only considering MS2
                             "missingValues"] += 1

        i += 1

    protein_columns = protein_columns.assign(missingValuesRate=(
        protein_columns["missingValues"] / i) * 100)
    
    protein_columns = protein_columns.query(
        "missingValuesRate < @missing_value_thresh")
    
    protein_columns = protein_columns.loc[:,
                                  protein_columns.columns.str.contains(name)]

    # filter the data_object with the remaining proteins/peptides names
    data_object[abundance_name] = protein_columns.merge(
        data_object[abundance_name])
    data_object[matrix_name] = protein_columns.merge(
        data_object[matrix_name])
    data_object[other_info_name] = protein_columns.merge(
        data_object[other_info_name])
    # In case there is mismatch between ID table and abundance table,
    # mannually remove the row with all NaN values
    # keep rows in data_object[abundance_name] where at least two values are 
    # not NaN(do this to all rows except the first row), otherwise can't
    # calculate the stdev
    if ignore_nan:
        data_object[abundance_name] = data_object[abundance_name].dropna(
            thresh=2, subset=data_object[abundance_name].columns[1:])
        # This will cause the veen diagram to be different from R program
    
    return data_object




In [85]:
def NormalizeToMedian(abundance_data, apply_log2=False):
    """_Normalizes each column by multiplying each value in that column with
    the median of all values in abundances (all experiments) and then dividing
    by the median of that column (experiment)._
    we find applying log2 transform first gives more robust results for PCA etc.
    See https://pubs.acs.org/doi/10.1021/acsomega.0c02564
    Args:
        abundance_data (_pd_): _description_
        apply_log2 (_bool_,): _apply log2 to all result_.
    Returns:
        _type_: _description_
        format:
         'abundances':        Accession  3_TrypsinLysConly_3A4_channel2
         A0A096LP49                    0.000000e+00
    """
    # all the columns/sample list
    columns = [col for col in abundance_data.select_dtypes(include=[
            np.number])]
    data_matrix = abundance_data[columns].values
    # replace 0 with nan
    data_matrix[data_matrix == 0] = np.nan
    medianOfAll = np.nanmedian(data_matrix)
    
    #normalize all median, all median/current run all protein median
    # apply log2 to all the values if apply_log2 is True
    if apply_log2:    
        for each_column in columns:
            abundance_data[each_column] = (
                np.log2(medianOfAll) * np.log2(abundance_data[each_column]) /
                np.log2(np.nanmedian(abundance_data[
                    each_column].replace(0, np.nan))))
    else:
        for each_column in columns:
            abundance_data[each_column] = (
                medianOfAll * abundance_data[each_column] /
                np.nanmedian(abundance_data[
                    each_column].replace(0, np.nan)))
    #TODO divide by zero error encountered in log2, temporarily set to 0
    abundance_data = abundance_data.replace([np.inf, -np.inf], 0)

    return abundance_data

In [86]:
def calculate_cvs(abundance_data):
    """_Calculate mean, stdev, cv for withn each protein/peptide abundance_

    Args:
        data_object (_type_): _full data frame_

    Returns:
        _type_: _df with Accession mean, stdev, cv for each protein/peptide_
    """
    if 'Accession' in abundance_data.columns:
        name = "Accession"
    if 'Annotated Sequence' in abundance_data.columns:
        name = "Annotated Sequence"
    abundance_data = abundance_data.assign(
        intensity=abundance_data.loc[:, ~abundance_data.columns.str.contains(
            name)].mean(axis=1, skipna=True),
        stdev=abundance_data.loc[:, ~abundance_data.columns.str.contains(
            name)].std(axis=1, skipna=True),
        CV=abundance_data.loc[:, ~abundance_data.columns.str.contains(name)].std(
            axis=1, skipna=True) / abundance_data.loc[
            :, ~abundance_data.columns.str.contains(name)].mean(
            axis=1, skipna=True) * 100)

    abundance_data = abundance_data.loc[:, [
            name, "intensity", "stdev", "CV"]]
    
    return abundance_data

In [87]:
def t_test_from_summary_stats(m1, m2, n1, n2, s1, s2, equal_var=False):
    """_Calculate T-test from summary using ttest_ind_from_stats from
    scipy.stats package_

    Args:
        m1 (_type_): _mean list of sample 1_
        m2 (_type_): mean list of sample 2_
        n1 (_type_): sample size list of sample 1_
        n2 (_type_): sample size list of sample 2_
        s1 (_type_): standard deviation list of sample 1_
        s2 (_type_): standard deviation list of sample 2_
        equal_var (_type_, optional): False would perform Welch's
        t-test, while set it to True would perform Student's t-test. Defaults
        to False.

    Returns:
        _type_: _list of P values_
    """

    p_values = []
    for i in range(len(m1)):
        _, benjamini = ttest_ind_from_stats(
            m1[i], s1[i], n1[i], m2[i], s2[i], n2[i], equal_var=equal_var)
        p_values.append(benjamini)

    return p_values

In [88]:
def impute_knn(abundance_data, k=5):
    """_inpute missing value from neighbor values_

    Args:
        abundance_data (_type_): _description_
        k (int, optional): _number of neighbors used_. Defaults to 5.
    Returns:
        _type_: _description_
        TODO: this knn imputer produces slightly different results (about 4%)
        from the one in R. Need to figure out why
    """
    name = abundance_data.columns[0]

    names = abundance_data[name]
    # x = abundance_data.select_dtypes(include=['float64', 'int64'])
    # imputer = KNNImputer(n_neighbors=k)
    # x_imputed = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)


    x = abundance_data.select_dtypes(include=['float', 'int'])
    imputer = KNNImputer(n_neighbors=k)
    x_imputed = imputer.fit_transform(x)
    x_imputed = pd.DataFrame(x_imputed, columns=x.columns)



    abundance_data.loc[:, x.columns] = x_imputed.values
    abundance_data[name] = names
    return abundance_data




In [89]:
def CalculatePCA(abundance_object, infotib,log2T = False):
    """_inpute PCA transformed and variance explained by each principal
    component_
    """
    name = abundance_object.columns[0]
    x = abundance_object
    
    sampleNames = x.columns[~x.columns.str.contains(
        name)].to_frame(index=False)

    if log2T: #apply log2 transformation
        x = np.log2(x.loc[:, ~x.columns.str.contains(name)].T.values)
    else:
        x = x.loc[:, ~x.columns.str.contains(name)].T.values
    # filter out columns with all zeros
    is_finite_col = np.isfinite(np.sum(x, axis=0))
    x_filtered = x[:, is_finite_col]

    
    # Instantiate PCA    
    pca = PCA()
    #
    # Determine transformed features
    #
    x_pca = pca.fit_transform(x_filtered)
    #
    # Determine explained variance using explained_variance_ration_ attribute
    #
    exp_var_pca = pca.explained_variance_ratio_
    #
    # Cumulative sum of eigenvalues; This will be used to create step plot
    # for visualizing the variance explained by each principal component.
    #
    cum_sum_eigenvalues = np.cumsum(exp_var_pca)
    #
    # convert numpy array to pandas dataframe for plotting
    
    pca_panda = pd.DataFrame(x_pca, columns=[
        'PC' + str(i+1) for i in range(x_pca.shape[1])])
    # add sample names to the dataframe
    pca_panda = pd.concat(
        [infotib, pca_panda], axis=1, join='inner')
    
    return pca_panda, exp_var_pca




In [90]:
def filter_by_name(data_dict, runname_list):
    """_Filter the data_dict based on runname_list, only keep the columns
    of the data_dict that are in the runname_list_
    Args:

    Returns:
        _type_: _description_
    """

    # make dict for each runname, no accession/sequence
    nameDict = dict(zip(data_dict["run_metadata"]["Run Names"],data_dict["run_metadata"]["Run Identifier"]))
    
    identifier_list = []
    
    identifier_list_plus = []
    if "Annotated Sequence" in runname_list:
        runname_list.remove("Annotated Sequence")
    if "Accession" in runname_list:
        runname_list.remove("Accession")
    for eachName in runname_list:
        identifier_list.append(nameDict[eachName])

    for eachName in runname_list:
        identifier_list_plus.append(nameDict[eachName])


    filtered_data = {}
   # filtered_data["meta"] = data_dict["meta"]
    runname_list.extend(["Annotated Sequence","Accession"])
    identifier_list_plus.extend(["Annotated Sequence","Accession"])

    #filtered_data["run_metadata"] = [item for item in data_dict[
    #   "run_metadata"] if item in runname_list]
    
    filtered_data["run_metadata"] = data_dict["run_metadata"][
        data_dict["run_metadata"]["Run Names"].isin(
            runname_list)]  
    filtered_data["protein_abundance"] = data_dict["protein_abundance"][[
        col for col in data_dict["protein_abundance"].columns if any(
            word == col for word in identifier_list_plus)]]
    filtered_data["peptide_abundance"] = data_dict["peptide_abundance"][[
        col for col in data_dict["peptide_abundance"].columns if any(
            word == col for word in identifier_list_plus)]]
    filtered_data["protein_other_info"] = data_dict["protein_other_info"][[
        col for col in data_dict["protein_other_info"].columns if any(
            word == col for word in identifier_list_plus)]]
    filtered_data["peptide_other_info"] = data_dict["peptide_other_info"][[
        col for col in data_dict["peptide_other_info"].columns if any(
            word == col for word in identifier_list_plus)]]
    filtered_data["protein_ID_matrix"] = data_dict["protein_ID_matrix"][[
        col for col in data_dict["protein_ID_matrix"].columns if any(
            word == col for word in identifier_list_plus)]]
    filtered_data["peptide_ID_matrix"] = data_dict["peptide_ID_matrix"][[
        col for col in data_dict["peptide_ID_matrix"].columns if any(
            word == col for word in identifier_list_plus)]]
    filtered_data["protein_ID_Summary"] = data_dict["protein_ID_Summary"][
        data_dict["protein_ID_Summary"]["Run Identifier"].isin(
            identifier_list)]
    filtered_data["peptide_ID_Summary"] = data_dict["peptide_ID_Summary"][
        data_dict["peptide_ID_Summary"]["Run Identifier"].isin(
            identifier_list)]
    return filtered_data

In [91]:
def ID_plots(data_object, plot_options, saved_settings, username=None):
    """_Prepare data for creating protein peptide identification bar
    plot_

    Args:
        data_dict (_type_): _description_
    """
    # Create an empty dictionary to store the group names and filters
    group_names = [key for key in saved_settings.keys() if "Order@" not in str(key)]

    # import the data and save order
    group_dict = {}

    if plot_options["ID mode"] == "MS2" or plot_options["ID mode"] == "total" or plot_options["ID mode"] == "stacked":
        x_axis_order = saved_settings["Order@Conditions"]
    elif plot_options["Group By X"] == "ID_Mode":
        print("ERROR: x axis separation of MS2/MBR not supported")
    else:
        x_axis_order = saved_settings["Order@"+plot_options["Group By X"]]
    if plot_options["ID mode"] == "grouped" or plot_options["ID mode"] == "grouped_stacked" and plot_options["Group By Color"] != "ID_Mode":
        color_order = saved_settings["Order@"+plot_options["Group By Color"]]
        plot_options["color_order"] = color_order

    plot_options["x_axis_order"] = x_axis_order

    # filter runs into different groups
    i = 1
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = saved_settings[eachGroup]["records"]

        group_dict[eachGroup] = filter_by_name(
            data_object,
            runname_sublist)  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1

        #print(group_dict[eachGroup]["run_metadata"])
    #display(data_object["protein_ID_Summary"])
    #display(group_dict[eachGroup]["protein_ID_Summary"])
    # create ID plots
    # allIDs table will be used to store all experiment name, ID types (
    # protein, peptide, MS2 and MS1 based), conditions and IDs numbers
    allIDs = pd.DataFrame(
        columns=["Run Identifier", "ID_Type", "ID_Mode", "Conditions", "IDs"])

    # loop through each group and extract IDs, put them into allIDs table
    for eachCondition in group_names:
        # Protein ID summary
        for index, row in group_dict[eachCondition][
                "protein_ID_Summary"].iterrows():
            for item in ["MS2_IDs",
                         "MBR_IDs",
                         "Total_IDs"]:
                if not pd.isna(group_dict[eachCondition][
                        "protein_ID_Summary"].at[index, item]):
                    # if the row with the item column is not empty,
                    # add it to allIDs table.
                    allIDs = pd.concat(
                        [allIDs,
                         pd.DataFrame(
                             [[group_dict[eachCondition][
                                 "protein_ID_Summary"].at[index, "Run Identifier"],
                              "protein",
                               item,
                               eachCondition,
                               group_dict[eachCondition][
                                 "protein_ID_Summary"].at[index, item]]],
                             columns=["Names",
                                      "ID_Type",
                                      "ID_Mode",
                                      "Conditions",
                                      "IDs"])],
                        ignore_index=True)
        # Peptide ID summary
        for index, row in group_dict[eachCondition][
                "peptide_ID_Summary"].iterrows():
            for item in ["MS2_IDs",
                         "MBR_IDs",
                         "Total_IDs"]:
                if not pd.isna(group_dict[eachCondition][
                        "peptide_ID_Summary"].at[index, item]):
                    allIDs = pd.concat(
                        [allIDs,
                         pd.DataFrame(
                             [[group_dict[eachCondition][
                                 "peptide_ID_Summary"].at[index, "Run Identifier"],
                              "peptide",
                               item,
                               eachCondition,
                               group_dict[eachCondition][
                                 "peptide_ID_Summary"].at[index, item]]],
                             columns=["Names",
                                      "ID_Type",
                                      "ID_Mode",
                                      "Conditions",
                                      "IDs"])],
                        ignore_index=True)
    # ######################allIDs format###################
    # name	ID_Type	ID_Mode	Conditions	IDs
    # file1	peptide	MS2_IDs	experimetn 1	xxxxx
    # file2	protein	MBR_IDs	experiment 2	xxxx
    # file3	peptide	Total_IDs	experiment 3	xxx
    #######################################################
    # Calcuate mean, standard deviation and number of replicates for each
    export_ids = allIDs.copy()
    # choose protein or peptide
    if plot_options["plot_type"] == "1":  # Protein ID
        allIDs = allIDs[allIDs["ID_Type"] == "protein"]
    elif plot_options["plot_type"] == "2":  # Peptide ID
        allIDs = allIDs[allIDs["ID_Type"] == "peptide"]

    # choose total, MS2 or stacked
    if plot_options["ID mode"] == "MS2":  # MS2 ID
        allIDs = allIDs[allIDs["ID_Mode"] == "MS2_IDs"]
    elif plot_options["ID mode"] == "total":
        # total ID combined, if not already summed (key exist), sum them
#         if allIDs[allIDs["ID_Mode"] == "Total_IDs"].empty:
#             grouped = allIDs.groupby('name').agg(
#                 {'IDs': 'sum', 'ID_Type': 'first', 'Conditions': 'first'})
#             grouped = grouped.reset_index()
#             grouped["ID_Mode"] = "Total_IDs"
#             allIDs = grouped
        allIDs = allIDs[allIDs["ID_Mode"] == "Total_IDs"]
    elif plot_options["Group By X"] == "ID_Mode" or plot_options["Group By Color"] == "ID_Mode" \
        or plot_options["Group By Stack"] == "ID_Mode" and not (plot_options["ID mode"] == "total" or plot_options["ID mode"] == "MS2"):  # total separated
        pass
    elif plot_options["ID mode"] == "MS2":
        allIDs = allIDs[allIDs["ID_Mode"] == "MS2_IDs"]
    else:
        allIDs = allIDs[allIDs["ID_Mode"] == "Total_IDs"]

    toPlotIDs = allIDs.groupby(["ID_Mode", "Conditions"]).agg({
        'IDs': ['mean', 'std', 'count'], 'ID_Type': 'first', })

    # rename the columns
    toPlotIDs.columns = ['IDs', 'stdev', 'n', 'ID_Type']
    # reset the index after grouping
    toPlotIDs = toPlotIDs.reset_index()
    # calculate the confidence interval based on 95%confidence interval`
    toPlotIDs["confInt"] = t.ppf(0.975, toPlotIDs['n']-1) * \
        toPlotIDs['stdev']/np.sqrt(toPlotIDs['n'])

    #add columns for the categories specified in settings file (the one with all the filenames)
    standard_groups = ["filter_in","filter_out","records"]
    categories = [col for col in list(saved_settings[list(group_names)[0]].keys()) if col not in standard_groups]
    for eachCategory in categories:
        toPlotIDs[eachCategory] = ""
        for eachGroup in group_names:
            toPlotIDs.loc[toPlotIDs["Conditions"]==eachGroup,eachCategory] = saved_settings[eachGroup][eachCategory]
    
    #display(toPlotIDs)
    fig = plot_IDChart_plotly(toPlotIDs,
                               username=username,
                               plot_options=plot_options)

    if WRITE_OUTPUT:    
        # export the data to csv for user downloading
        data_dir = os.path.join(APPFOLDER, "csv/")
        # create the directory if it does not exist
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        categories = [col for col in list(saved_settings[list(group_names)[0]].keys()) if col not in standard_groups]
        
        for eachCategory in categories:
            export_ids[eachCategory] = ""
            for eachGroup in group_names:
                export_ids.loc[export_ids["Conditions"]==eachGroup,eachCategory] = saved_settings[eachGroup][eachCategory]
        export_ids = export_ids.replace({"Names": dict(zip(data_object["run_metadata"]["Run Identifier"],data_object["run_metadata"]["Run Names"]))})
        
        # export the data to csv
        export_ids.to_csv(os.path.join(
            data_dir, f"{username}_ID_data.csv"), index=False)
        
        print("Downloading links...")

        # create the link for downloading the data
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_ID_data.csv"

        # add SVG download link

        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_ID_Bar_Plot.svg"

        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_ID_Bar_Plot.svg"), format = "svg", validate = False, engine = "kaleido")


    else:
        CSV_link = None
        SVG_link = None
    return fig, CSV_link, SVG_link

def plot_IDChart_plotly(ID_data,
                        username=None,
                        plot_options=None):
    """_Plot the ID bar plot for the given data_

    Args:
        ID_data (_type_): _description_
        username (str, optional): _description_. Defaults to "test".
        plot_options (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """

    plot_div = None
    

    if plot_options["ID mode"] == "grouped":  
        # plot options
        # error bar
        if plot_options["error bar"] == "stdev":
            error_bars = "stdev"
            error_visibile = True
        elif plot_options["error bar"] == "ci95":
            error_bars = "confInt"
            error_visibile = True
        else:
            error_bars = "stdev"
            error_visibile = False

        # mean label
        if plot_options["mean label"] == "True" or \
                plot_options["mean label"] == True:
            total_labels = [{"x": x, "y": total*1.15, "text": str(
                int(total)), "showarrow": False} for x, total in zip(
                    ID_data["Conditions"], ID_data["IDs"])]
        else:
            total_labels = []   # no mean labels

        if plot_options["Group By X"] == "ID_Mode" or plot_options["Group By Color"] == "ID_Mode":  # total separated
            ID_data = ID_data[ID_data["ID_Mode"] != "Total_IDs"]
        else:
            ID_data = ID_data[ID_data["ID_Mode"] == "Total_IDs"]
        #find out present categories
        categories = plot_options["color_order"]
        # create the plot
        fig_data = []
        i = 0
        for eachCategory in categories:
            fig_data.append(go.Bar(name = eachCategory,
                        x=ID_data.loc[ID_data[plot_options["Group By Color"]]==eachCategory,plot_options["Group By X"]].tolist(),
                        y=ID_data.loc[ID_data[plot_options["Group By Color"]]==eachCategory,"IDs"].tolist(),
                        marker_color = plot_options["color"][i],
                        text = [round(x) for x in ID_data.loc[ID_data[plot_options["Group By Color"]]==eachCategory,"IDs"].tolist()],
                        error_y=dict(
                            type = "data",
                            array = ID_data.loc[ID_data[plot_options["Group By Color"]]==eachCategory,error_bars].tolist(),
                            visible = error_visibile
                        )))
            i = i + 1                    

        fig = go.Figure(data = fig_data,
                        layout=go.Layout(yaxis_title=plot_options["Y Title"],
                        xaxis_title=plot_options["Group By X"],
                        barmode="group",paper_bgcolor="rgba(255,255,255,255)",
                        plot_bgcolor="rgba(255, 255, 255, 255)",
                        yaxis=dict(showline=True, linewidth=1, linecolor='black'),
                        xaxis=dict(showline=True, linewidth=1, linecolor='black')))
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])

    elif plot_options["ID mode"] == "stacked":
        # plot options
        # error bar
        if plot_options["error bar"] == "stdev":
            error_bars = "stdev"
            error_visibile = True
        elif plot_options["error bar"] == "ci95":
            error_bars = "confInt"
            error_visibile = True
        else:
            error_bars = "stdev"
            error_visibile = False

        # mean label
        if plot_options["mean label"] == "True" or \
                plot_options["mean label"] == True:
            total_labels = [{"x": x, "y": total*1.15, "text": str(
                int(total)), "showarrow": False} for x, total in zip(
                    ID_data["Conditions"], ID_data["IDs"])]
        else:
            total_labels = []   # no mean labels
        if plot_options["Group By X"] == "ID_Mode" or plot_options["Group By Stack"] == "ID_Mode":  # total separated
            ID_data = ID_data[ID_data["ID_Mode"] != "Total_IDs"]
        else:
            ID_data = ID_data[ID_data["ID_Mode"] == "Total_IDs"]

        if plot_options["Group By Stack"] == "ID_Mode":
            layers = ["MS2_IDs", "MBR_IDs"]
        else:
            layers = ID_data.groupby(plot_options["Group By Stack"]).first().reset_index()[plot_options["Group By Stack"]].tolist()
        fig_data = []
        last_layer = None
        i = 0
        for eachLayer in layers: 
            if last_layer == None:
                fig_data.append(go.Bar(
                    name = eachLayer,
                    x = ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),plot_options["Group By X"]].tolist(),
                    y = ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist(),
                    marker_color = plot_options["color"][i],
                    text = [round(x) for x in ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist()],
                    error_y= dict(
                        type = "data",
                        array = ID_data.loc[ID_data[plot_options["Group By Stack"]]==eachLayer,error_bars].tolist(),
                        visible = error_visibile
                    )
                ))
                bases = ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"]
                i = i + 1
                
            else:
                fig_data.append(go.Bar(
                    name = eachLayer,
                    x = ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),plot_options["Group By X"]].tolist(),
                    y = ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist(),
                    base=bases,
                    marker_color = plot_options["color"][i],
                    opacity=0.5,
                    text = [round(x) for x in bases + ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist()],
                    error_y= dict(
                        type = "data",
                        array = ID_data.loc[ID_data[plot_options["Group By Stack"]]==eachLayer,error_bars].tolist(),
                        visible = error_visibile
                    )
                ))
                print(bases)
                bases = bases + ID_data.loc[(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist()
            last_layer = eachLayer
        fig = go.Figure(
                data = fig_data,
                layout=go.Layout(
                yaxis_title=plot_options["Y Title"],
                xaxis_title=plot_options["Group By X"],
                barmode="stack", 
                paper_bgcolor="rgba(255,255,255,255)",
                plot_bgcolor="rgba(255, 255, 255, 255)",
                yaxis=dict(showline=True, linewidth=1, linecolor='black'),
                xaxis=dict(showline=True, linewidth=1, linecolor='black')
            ))          
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])            
    elif plot_options["ID mode"] == "grouped_stacked":
        # plot options
        # error bar
        if plot_options["error bar"] == "stdev":
            error_bars = "stdev"
            error_visibile = True
        elif plot_options["error bar"] == "ci95":
            error_bars = "confInt"
            error_visibile = True
        else:
            error_bars = "stdev"
            error_visibile = False

        # mean label
        if plot_options["mean label"] == "True" or \
                plot_options["mean label"] == True:
            total_labels = [{"x": x, "y": total*1.15, "text": str(
                int(total)), "showarrow": False} for x, total in zip(
                    ID_data["Conditions"], ID_data["IDs"])]
        else:
            total_labels = []   # no mean labels

        if plot_options["Group By X"] == "ID_Mode" or plot_options["Group By Color"] == "ID_Mode"or plot_options["Group By Stack"] == "ID_Mode":  # total separated
            ID_data = ID_data[ID_data["ID_Mode"] != "Total_IDs"]
        else:
            ID_data = ID_data[ID_data["ID_Mode"] == "Total_IDs"]

        #make data tidy
        if plot_options["Group By Stack"] == "ID_Mode":
            layers = ["MS2_IDs", "MBR_IDs"]
        else:
            layers = ID_data.groupby(plot_options["Group By Stack"]).first().reset_index()[plot_options["Group By Stack"]].tolist()
        categories = plot_options["color_order"]
        
        fig_data = []
        i = 0
        for eachCategory in categories:
            last_layer = None
            j = 0
            for eachLayer in layers: 
                if last_layer == None:
                    fig_data.append(go.Bar(
                        name = str(eachLayer) + " " + str(eachCategory),
                        x = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),plot_options["Group By X"]],
                        y = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"],
                        offsetgroup=i,
                        text = [round(x) for x in ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist()],
                        marker_color = plot_options["color"][i],
                        error_y = dict(
                            type = "data",
                            array = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),error_bars],
                            visible=True)
                    ))
                    bases = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"]
                else:

                    fig_data.append(go.Bar(
                        name = str(eachLayer) + " " + str(eachCategory),
                        x = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),plot_options["Group By X"]],
                        y = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"],
                        base=bases,
                        offsetgroup=i,
                        text = [round(x) for x in ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist()+bases],
                        marker_color = plot_options["color"][i],
                        opacity=1/2**j,
                        error_y = dict(
                            type = "data",
                            array = ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),error_bars],
                            visible=True)
                        ))
                    bases = bases + ID_data.loc[(ID_data[plot_options["Group By Color"]]==eachCategory)&(ID_data[plot_options["Group By Stack"]]==eachLayer),"IDs"].tolist()
                last_layer=eachLayer
                j = j + 1
            i = i + 1

        fig = go.Figure(
            fig_data,
            layout=go.Layout(
                yaxis_title=plot_options["Y Title"],
                xaxis_title=plot_options["Group By X"],
                barmode="group",
                plot_bgcolor="rgba(255, 255, 255, 255)",
                paper_bgcolor="rgba(255, 255, 255, 255)",
                yaxis=dict(showline=True, linewidth=1, linecolor='black'),
                xaxis=dict(showline=True, linewidth=1, linecolor='black')
            )
        )
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])
    else:

        # plot options
        # error bar
        if plot_options["error bar"] == "stdev":
            error_bars = "stdev"
        elif plot_options["error bar"] == "ci95":
            error_bars = "confInt"
        else:
            error_bars = None

        # mean label
        if plot_options["mean label"] == "True" or \
                plot_options["mean label"] == True:
            total_labels = [{"x": x, "y": total*1.15, "text": str(
                int(total)), "showarrow": False} for x, total in zip(
                    ID_data["Conditions"], ID_data["IDs"])]
        else:
            total_labels = []   # no mean labels

        # create the plot
        fig = px.bar(ID_data,
                     x="Conditions",
                     y="IDs",
                     error_y=error_bars,
                     color="Conditions",
                     color_discrete_sequence=plot_options["color"],
                     width=plot_options["width"],
                     height=plot_options["height"],
                     )
        fig.update_layout(xaxis_title=plot_options["X Title"],
                          yaxis_title=plot_options["Y Title"],
                          annotations=total_labels,
                          font=plot_options["font"],
                          plot_bgcolor ="rgba(255, 255, 255, 255)",
                          paper_bgcolor="rgba(255, 255, 255, 255)",
                          yaxis=dict(showline=True, linewidth=1, linecolor='black'),
                          xaxis=dict(showline=True, linewidth=1, linecolor='black')
                          )
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])
    return fig

In [92]:
# CV Violin plots ###
def CV_plots(data_object, plot_options, saved_settings, username=None):
    """_Prepare data for creating protein CV violin plots_
    """
    group_names = [key for key in saved_settings.keys() if "Order@" not in str(key)]
    
    # import the data and save order
    group_dict = {}

    if plot_options["CV mode"] == "MS2" or plot_options["CV mode"] == "total" or plot_options["CV mode"] == "stacked":
        x_axis_order = saved_settings["Order@Conditions"]
    else:
        x_axis_order = saved_settings["Order@"+plot_options["Group By X"]]
    plot_options["x_axis_order"] = x_axis_order
    if plot_options["CV mode"] == "grouped" or plot_options["CV mode"] == "grouped_stacked" and plot_options["Group By Color"] != "ID_Mode":
        color_order = saved_settings["Order@"+plot_options["Group By Color"]]
        plot_options["color_order"] = color_order
    # filter runs into different groups
    i = 1
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = saved_settings[eachGroup]["records"]

        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1

    if plot_options["plot_type"] == 1:
        matrix_name = "protein_abundance"
    elif plot_options["plot_type"] == 2:
        matrix_name = "peptide_abundance" 

    # create a dictionary to store the intensity data
    Intensity_dict = {}
    if plot_options["Group By X"] == "ID_Mode"or plot_options["Group By Color"] == "ID_Mode" \
        or plot_options["Group By Stack"] == "ID_Mode"and not (plot_options["CV mode"] == "total" or plot_options["CV mode"] == "MS2"):  # total separated
        Intensity_dict_MS2 = {}
        for eachGroup in group_names:
            current_condition_data = filter_by_missing_values(
                group_dict[eachGroup])
            Intensity_dict[eachGroup] = NormalizeToMedian(
                current_condition_data[matrix_name])
            current_condition_data_MS2 = filter_by_missing_values_MS2( #returns
                group_dict[eachGroup])
            Intensity_dict_MS2[eachGroup] = NormalizeToMedian(
                current_condition_data_MS2[matrix_name])
    elif plot_options["CV mode"] == "MS2":
        Intensity_dict_MS2 = {}
        for eachGroup in group_names:
            current_condition_data_MS2 = filter_by_missing_values_MS2(
                group_dict[eachGroup])
            Intensity_dict[eachGroup] = NormalizeToMedian(
                current_condition_data_MS2[matrix_name])
    else:
        for eachGroup in group_names:
            current_condition_data = filter_by_missing_values(
                group_dict[eachGroup])
            Intensity_dict[eachGroup] = NormalizeToMedian(
                current_condition_data[matrix_name])

    all_cvs = pd.DataFrame()

    for eachGroup in Intensity_dict:
        current = calculate_cvs(
            Intensity_dict[eachGroup]).assign(Conditions=eachGroup,ID_Mode="All IDs")
        all_cvs = pd.concat([all_cvs, current], ignore_index=True)
    if plot_options["Group By X"] == "ID_Mode"or plot_options["Group By Color"] == "ID_Mode"or plot_options["Group By Stack"] == "ID_Mode"and not plot_options["CV mode"] == "total":  # total separated  # total separated
        print("MS2 CVs...")
        for eachGroup in Intensity_dict_MS2:
            current = calculate_cvs(
                Intensity_dict_MS2[eachGroup]).assign(Conditions=eachGroup,ID_Mode="MS2 IDs")
            all_cvs = pd.concat([all_cvs, current], ignore_index=True)

    #add columns for the categories specified in settings file (the one with all the filenames)
    standard_groups = ["filter_in","filter_out","records"]
    categories = [col for col in list(saved_settings[list(group_names)[0]].keys()) if col not in standard_groups]
    for eachCategory in categories:
        all_cvs[eachCategory] = ""
        for eachGroup in group_names:
            all_cvs.loc[all_cvs["Conditions"]==eachGroup,eachCategory] = saved_settings[eachGroup][eachCategory]


    # ######################all_CVs format###################
#      Accession     intensity          stdev          CV   Conditions
# 0       A6NHR9  3.248547e+06  672989.819300   20.716643    DDMandDTT
# 1       A8MTJ3  5.031539e+05  195535.383583   38.861944    DDMandDTT
# 2       E9PAV3  5.330290e+05  161385.491163   30.277056    DDMandDT
    #######################################################

    return plot_CV_violin(allCVs=all_cvs,
                          username=username,
                          plot_options=plot_options)


def plot_CV_violin(allCVs,
                   username=None,
                   plot_options=None,
                   ):
    """_Plot the CV violin plot for the given data._

    Args:
        allCVs (_type_): _description_
        username (_type_, optional): _description_. Defaults to None.
        plot_options (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    plot_div = None
    CSV_link = None
    SVG_link = None

    group_names = [key for key in saved_settings.keys() if "Order@" not in str(key)]

    allCVs_summary = allCVs.groupby(["Conditions"]).agg(
        {'CV': ['median', 'mean']}).reset_index()
    allCVs_summary["ID_Mode"] = "All IDs"
    temp = allCVs[allCVs["ID_Mode"]=="MS2 IDs"].groupby(["Conditions"]).agg(
        {'CV': ['median', 'mean']}).reset_index()
    temp["ID_Mode"] = "MS2 IDs"
    allCVs_summary = pd.concat([temp,allCVs_summary])
    allCVs_summary.columns = ["Conditions", 'meds', 'CoVar',"ID_Mode"]

    
    standard_groups = ["filter_in","filter_out","records"]
    categories = [col for col in list(saved_settings[group_names[0]].keys()) if col not in standard_groups]
    for eachCategory in categories:
        allCVs_summary[eachCategory] = ""
        for eachGroup in group_names:
            allCVs_summary.loc[allCVs_summary["Conditions"]==eachGroup,eachCategory] = saved_settings[eachGroup][eachCategory]


    if plot_options["CV mode"] == "grouped":  
        #find out present categories
        categories = plot_options["color_order"] 
        # create the plot
        fig_data = []
        #display(ID_data)
        i = 0
        for eachCategory in categories:
            fig_data.append(go.Violin(name = str(eachCategory),
                        x=allCVs.loc[allCVs[plot_options["Group By Color"]]==eachCategory,
                        plot_options["Group By X"]].tolist(),
                        y=allCVs.loc[allCVs[plot_options["Group By Color"]]==eachCategory,"CV"].tolist(),
                        fillcolor = plot_options["color"][i],                 
                        box=dict(visible=bool(plot_options["box"]))
                        ))
            i = i + 1
        
                        
        fig = go.Figure(data = fig_data)
        
        fig.update_layout(violinmode='group')
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])
    elif plot_options["CV mode"] == "stacked":  
            
        #find out present categories
        layers = allCVs.groupby(plot_options["Group By Stack"]).first().reset_index()[plot_options["Group By Stack"]].tolist()
        # create the plot
        fig_data = []
        #display(ID_data)
        i = 0

        for eachLayer in layers:
            fig_data.append(go.Violin(name = eachLayer,
                        x=allCVs.loc[allCVs[plot_options["Group By Stack"]]==eachLayer,plot_options["Group By X"]].tolist(),
                        y=allCVs.loc[allCVs[plot_options["Group By Stack"]]==eachLayer,"CV"].tolist(),
                        box=dict(visible=bool(plot_options["box"])),
                        fillcolor = plot_options["color"][i]
                        ))
            i = i + 1
        fig = go.Figure(data = fig_data)
        
        fig.update_layout(violinmode='overlay',
                          plot_bgcolor='white',
                          paper_bgcolor='white',
                          yaxis=dict(showline=True, linewidth=1, linecolor='black'),
                          xaxis=dict(showline=True, linewidth=1, linecolor='black')
        )  
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])

    elif plot_options["CV mode"] == "grouped_stacked":
        #make data tidy
        layers = allCVs.groupby(plot_options["Group By Stack"]).first().reset_index()[plot_options["Group By Stack"]].tolist()
        categories = plot_options["color_order"] 
        fig_data = []
        i = 0
        j = 0
        for eachCategory in categories:
            for eachLayer in layers:
                fig_data.append(go.Violin(
                    name = str(eachLayer) + " " + str(eachCategory),
                    x = allCVs.loc[(allCVs[plot_options["Group By Color"]]==eachCategory)&(allCVs[plot_options["Group By Stack"]]==eachLayer),plot_options["Group By X"]],
                    y = allCVs.loc[(allCVs[plot_options["Group By Color"]]==eachCategory)&(allCVs[plot_options["Group By Stack"]]==eachLayer),"CV"],
                    box=dict(visible=bool(plot_options["box"])),
                    offsetgroup=i,
                    fillcolor = plot_options["color"][j]
                    ))
                j = j + 1
            i = i + 1

        fig = go.Figure(
            fig_data,
            layout=go.Layout(
                yaxis_title=plot_options["Y Title"],
                xaxis_title=plot_options["Group By X"],
                violinmode="group",
                plot_bgcolor='white',
                paper_bgcolor='white',
                yaxis=dict(showline=True, linewidth=1, linecolor='black'),
                xaxis=dict(showline=True, linewidth=1, linecolor='black')
            )
        )
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])
    else:
    # create the interactive plot
        # median label
        if plot_options["median label"] == "True" or \
                plot_options["median label"] == True:
            total_labels = [{"x": x, "y": total*1.15, "text": str(
                round(total,1)), "showarrow": False} for x, total in zip(
                allCVs_summary["Conditions"], allCVs_summary["meds"])]
        else:
            total_labels = []   # no median labels
        fig = px.violin(allCVs,
                        x="Conditions",
                        y='CV',
                        color="Conditions",
                        box=bool(plot_options["box"]),
                        hover_data=["Conditions", 'CV'],
                        color_discrete_sequence=plot_options["color"],
                        width=plot_options["width"],
                        height=plot_options["height"],
                        )

        fig.update_layout(
            yaxis=dict(title=plot_options["Y Title"],
                    range=plot_options["ylimits"], showline=True, linewidth=1, linecolor='black'),
            font=plot_options["font"],
            xaxis=dict(title=plot_options["X Title"], showline=True, linewidth=1, linecolor='black'),
            showlegend=True,
            annotations=total_labels,
            plot_bgcolor='white',
            paper_bgcolor='white',
            )
        fig.update_xaxes(categoryorder='array', categoryarray = plot_options["x_axis_order"])

    if WRITE_OUTPUT:        
        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_CV_Violin_Plot.svg"), format = "svg", validate = False, engine = "kaleido")
        
        # create the download CSV and its link
        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        allCVs.to_csv(os.path.join(
            data_dir, f"{username}_all_CV.csv"), index=False)
        allCVs_summary.to_csv(os.path.join(
            data_dir, f"{username}_CV_summary.csv"), index=False)
        print("Downloading links...")
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_all_CV.csv"

        # download SVG link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_CV_Violin_Plot.svg"


    return fig, CSV_link, SVG_link




In [93]:
def venns_plots(data_object, plot_options, saved_settings, username=None,missing_values_max=33):
    """_Prepare data for creating ID veens plots (up to three groups)_
    """
    group_names = []

    # no compare groups is provided, compare first two
    
    for each_key in plot_options["compare groups"]:
        if each_key in plot_options["compare groups"] and saved_settings["Order@Conditions"]:
            group_names.append(each_key)
    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = saved_settings[eachGroup]["records"]
        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1
    data_set = []
    labels_set = []
    if plot_options["plot_type"] == 1:
        matrix_name = "protein_abundance"
        molecule_name = "Accession"
        is_protein = True
    elif plot_options["plot_type"] == 2:
        matrix_name = "peptide_abundance"
        molecule_name = "Annotated Sequence"
        is_protein = False
    
    for eachGroup in group_names:
        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup], is_protein=is_protein,missing_value_thresh=missing_values_max)

        data_set.append(
            set(current_condition_data[matrix_name][molecule_name].unique()))
        labels_set.append(eachGroup)

    #print(data_set)

    fig = venn_to_plotly(
        data_set,
        labels_set,
        plot_options=plot_options,
        username=username)
    CSV_link = None
    SVG_link = None

    if WRITE_OUTPUT:
        print("Downloading links...")
        # SVG file link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_ID_venns_Plot.svg"

        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_ID_venns_Plot.svg"), format = "svg", validate = False, engine = "kaleido")
        
        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        i = 0
        for eachSet in data_set:
            pd.DataFrame({"Accession": list(eachSet)}).to_csv(os.path.join(
                data_dir, f"{username}_{labels_set[i]}_Venn.csv"), index=False)
            i = i + 1
    
    return fig, SVG_link, CSV_link

def venn_to_plotly(L_sets,
                   L_labels=None,
                   plot_options=None,
                   username=None):
    """_Creates a venn diagramm from a list of
    sets and returns a plotly figure_
    """
    
    # get number of sets
    n_sets = len(L_sets)

    # choose and create matplotlib venn diagramm
    if n_sets == 2:
        if L_labels and len(L_labels) == n_sets:
            v = venn2(L_sets, L_labels)
        else:
            v = venn2(L_sets)
    elif n_sets == 3:
        if L_labels and len(L_labels) == n_sets:
            v = venn3(L_sets, L_labels)
        else:
            v = venn3(L_sets)
    # supress output of venn diagramm
    # plt.show()
    plt.close()

    # Create empty lists to hold shapes and annotations
    L_shapes = []
    L_annotation = []

    # Define color list for sets
    L_color = plot_options["color"]

    # Create empty list to make hold of min and max values of set shapes
    L_x_max = []
    L_y_max = []
    L_x_min = []
    L_y_min = []

    for i in range(0, n_sets):

        # create circle shape for current set

        shape = go.layout.Shape(
            type="circle",
            xref="x",
            yref="y",
            x0=v.centers[i][0] - v.radii[i],
            y0=v.centers[i][1] - v.radii[i],
            x1=v.centers[i][0] + v.radii[i],
            y1=v.centers[i][1] + v.radii[i],
            fillcolor=L_color[i],
            line_color=L_color[i],
            opacity=plot_options["opacity"]
        )

        L_shapes.append(shape)

        # create set label for current set
        try:
            anno_set_label = go.layout.Annotation(
                xref="x",
                yref="y",
                x=v.set_labels[i].get_position()[0],
                y=v.set_labels[i].get_position()[1],
                text=v.set_labels[i].get_text(),
                showarrow=False
            )

            L_annotation.append(anno_set_label)

            # get min and max values of current set shape
            L_x_max.append(v.centers[i][0] + v.radii[i])
            L_x_min.append(v.centers[i][0] - v.radii[i])
            L_y_max.append(v.centers[i][1] + v.radii[i])
            L_y_min.append(v.centers[i][1] - v.radii[i])
        except Exception as err:
            print(f"No set labels found {err}")

    # determine number of subsets
    n_subsets = sum([scipy.special.binom(n_sets, i+1)
                     for i in range(0, n_sets)])

    for i in range(0, int(n_subsets)):
        try:

            # create subset label (number of common elements for current subset

            anno_subset_label = go.layout.Annotation(
                xref="x",
                yref="y",
                x=v.subset_labels[i].get_position()[0],
                y=v.subset_labels[i].get_position()[1],
                text=v.subset_labels[i].get_text(),
                showarrow=False
            )

            L_annotation.append(anno_subset_label)
        except Exception as err:
            print(f"No set labels found {err}")
    # define off_set for the figure range
    off_set = 0.2

    # get min and max for x and y dimension to set the figure range
    x_max = max(L_x_max) + off_set
    x_min = min(L_x_min) - off_set
    y_max = max(L_y_max) + off_set
    y_min = min(L_y_min) - off_set

    # create plotly figure

    fig = go.Figure()

    # set xaxes range and hide ticks and ticklabels
    fig.update_xaxes(
        range=[x_min, x_max],
        showticklabels=False,
        ticklen=0
    )

    # set yaxes range and hide ticks and ticklabels
    fig.update_yaxes(
        range=[y_min, y_max],
        scaleanchor="x",
        scaleratio=1,
        showticklabels=False,
        ticklen=0
    )

    # set figure properties and add shapes and annotations
    fig.update_layout(
        plot_bgcolor='white',
        margin=dict(b=0, l=10, pad=0, r=10, t=40),
        width=800,
        height=400,
        shapes=L_shapes,
        annotations=L_annotation,
        title=dict(text=plot_options["title"], x=0.5, xanchor='center')
    )
    

    return fig


In [94]:
# ###Volcano plots####
def volcano_plots(data_object,  plot_options, saved_settings, username=None, missing_values_max=33):
    """_Prepare data for creating intensity volcano plots (two groups)_
    """
    group_names = []

    # no compare groups is provided, compare first two
    for each_key in plot_options["compare groups"]:
        if each_key in plot_options["compare groups"] and saved_settings["Order@Conditions"]:
            group_names.append(each_key)

    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = saved_settings[eachGroup]["records"]

        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1
    # create a dictionary to store the intensity data
    Intensity_dict = {}

    for eachGroup in group_names:
        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup], missing_value_thresh=missing_values_max)
        Intensity_dict[eachGroup] =  current_condition_data["protein_abundance"]
        
        
    group1 = group_names[0]
    group2 = group_names[1]
    # calculate mean, standard deviation, and the number of non-null
    # elements for each row/protein
    group1Data = (Intensity_dict[group1]
                  .assign(**{group1+'_Intensity': Intensity_dict[group1].drop(
        columns=['Accession']).mean(axis=1),
        "group1_stdev":Intensity_dict[group1].drop(
                      columns=['Accession']).std(axis=1),
        "group1_num":Intensity_dict[group1].drop(
                      columns=['Accession']).shape[1] - Intensity_dict[
                      group1].isna().sum(axis=1)})
                  .loc[:, [group1+'_Intensity',
                           'group1_stdev',
                           'group1_num',
                           'Accession']])
    """ group1Data
            group1_Intensity  group1_stdev  group1_num   Accession
    0        2.824766e+05  1.708060e+05          15  A0A0B4J2D5
    1        2.650998e+06  6.259645e+05          15      A2RUR9
    2        1.973150e+05  5.645698e+04          15      A8MTJ3
    3        2.524020e+05  1.355699e+05          15      A8MWD9
    """
    group1Prots = group1Data.loc[:, ['Accession']]

    group2Data = (Intensity_dict[group2]
                  .assign(**{group2+'_Intensity': Intensity_dict[group2].drop(
        columns=['Accession']).mean(axis=1),
        "group2_stdev":Intensity_dict[group2].drop(
                      columns=['Accession']).std(axis=1),
        "group2_num":Intensity_dict[group2].drop(
                      columns=['Accession']).shape[1] - Intensity_dict[
                      group2].isna().sum(axis=1)})
                  .loc[:, [group2+'_Intensity',
                           'group2_stdev',
                           'group2_num',
                           'Accession']])
    # find common proteins
    commonProts = (group2Data.loc[:, ['Accession']]
                   .merge(group1Prots, on='Accession', how='inner'))
    # only leave common proteins
    group2Data = (group2Data
                  .merge(commonProts, on='Accession', how='inner'))
    group1Data = (group1Data
                  .merge(commonProts, on='Accession', how='inner'))

    group2Median = group2Data[group2+'_Intensity'].median(
        )
    group1Median = group1Data[group1+'_Intensity'].median(
        )
    #this is the median for each protein across both groups, you could do a ratio, but this puts it in terms of
    allmedian = pd.DataFrame({"col2":group2Data[group2+'_Intensity'],"col1":group1Data[group1+'_Intensity']}).median(axis=1,numeric_only=True)
    
    if (Intensity_dict[group1].shape[1] > 3 and
        Intensity_dict[group2].shape[1] > 3 and
            group2 != group1):
        # calculate the ratio between two group median,
        # will be used to normalize them
        ratio1 = allmedian / group1Median
        ratio2 = allmedian / group2Median

        # merge these two set of data together, adjust groups with ratio to 
        # median of all. Calculate pOriginal, p, significant
        # pOriginal is a numpy array or list of p-values
        # method is the method to be used for adjusting the p-values
        volcanoData = (group2Data
                       .merge(group1Data, on='Accession', how='inner'))

        volcanoData = (volcanoData
                       .assign(**{group1+'_Intensity':lambda x: volcanoData[
                           group1+'_Intensity'] * ratio1}))
        volcanoData = (volcanoData
                       .assign(**{group2+'_Intensity':lambda x: volcanoData[
                           group2+'_Intensity'] * ratio2}))

        volcanoData = (volcanoData
                       .assign(
                           pOriginal=t_test_from_summary_stats(
                               m1=volcanoData[group2+'_Intensity'],
                               m2=volcanoData[group1+'_Intensity'],
                               s1=volcanoData['group2_stdev'],
                               s2=volcanoData['group1_stdev'],
                               n1=volcanoData['group2_num'],
                               n2=volcanoData['group1_num'])))
        # filter out rows in volcanoData that have pOriginal == nan
        # if pOriginal is nan, then the p value will be nan
        volcanoData = volcanoData[volcanoData['pOriginal'].notna()]
        volcanoData = (volcanoData
                       .assign(benjamini=multipletests(volcanoData[
                           "pOriginal"], method='fdr_bh')[1]))

        volcanoData = (volcanoData
                       .assign(significant=volcanoData['benjamini'] < 0.01))

        # add upRegulated, downRegulated, and notRegulated columns
        volcanoData = volcanoData.assign(upRegulated=lambda x: (
            np.log10(volcanoData[group2+'_Intensity']/volcanoData[
                group1+'_Intensity']) > 0.05) & (volcanoData['significant']))

        volcanoData = volcanoData.assign(downRegulated=lambda x: (
            np.log10(volcanoData[group2+'_Intensity']/volcanoData[
                group1+'_Intensity']) < -0.05) & (volcanoData['significant']))
        volcanoData = volcanoData.assign(notRegulated=lambda x: (abs(
           np.log10(volcanoData[group2+'_Intensity']/volcanoData[
                group1+'_Intensity'])) <= 0.05) & (~volcanoData['significant']))
        fig = plot_volcano_colored(
            volcanoData,
            label=f"({group2}/{group1})",
            plot_options=plot_options,
            username=username,
        )
        CSV_link = None
        SVG_link = None
        if WRITE_OUTPUT:
            # create the file for donwnload
            img_dir = os.path.join(APPFOLDER, "images/")
            if not os.path.exists(img_dir):
                Path(img_dir).mkdir(parents=True)

            fig.write_image(os.path.join(
                img_dir, f"{username}_abundance_volcano_Plot.svg"), format = "svg", validate = False, engine = "kaleido")
            # create the download CSV and its link

            data_dir = os.path.join(APPFOLDER, "csv/")
            if not os.path.exists(data_dir):
                Path(data_dir).mkdir(parents=True)
            volcanoData.to_csv(os.path.join(
                data_dir, f"{username}_up_down_regulated_volcano.csv"),
                index=False)
            print("Downloading links...")
            CSV_link = f"/files/{url_base}/csv/" \
                f"{username}_up_down_regulated_volcano.csv"

            # download SVG link
            SVG_link = f"/files/{url_base}/images/" \
                f"{username}_abundance_volcano_Plot.svg"
        
        return fig, CSV_link, SVG_link


def plot_volcano_colored(allData,
                         label,
                         plot_options=None,
                         username=None,):
    group_names = []

    # no compare groups is provided, compare first two
    for each_key in plot_options["compare groups"]:
        if each_key in plot_options["compare groups"] and saved_settings["Order@Conditions"]:
            group_names.append(each_key)
    
    total_labels = []
    left = group_names[0]+'_Intensity'
    right = group_names[1]+'_Intensity'
    downData = allData[allData['downRegulated']
                       == True]
    upData = allData[allData['upRegulated'] == True]

    fig = px.scatter(
        width=plot_options["width"],
        height=plot_options["height"],)
    if allData.shape[0] != 0:
        fig.add_scatter(x=np.log2(allData[right]/allData[left]),
                        y=-np.log10(allData["benjamini"]),
                        text=allData["Accession"],
                        mode="markers", marker=dict(
                            color=plot_options["all color"]))
    if downData.shape[0] != 0:
        fig.add_scatter(x=np.log2(downData[right]/downData[left]),
                        y=-np.log10(downData["benjamini"]),
                        text=downData["Accession"],
                        mode="markers",
                        marker=dict(color=plot_options["down color"]))
    if upData.shape[0] != 0:
        fig.add_scatter(x=np.log2(upData[right]/upData[left]),
                        y=-np.log10(upData["benjamini"]),
                        text=upData["Accession"],
                        mode="markers",
                        marker=dict(color=plot_options["up color"]))
        fig.update_traces(
            mode="markers",
            hovertemplate="%{text}<br>x=: %{x}"
            " <br>y=: %{y}")
    fig.add_hline(y=2)
    fig.add_vline(x=-np.log2(10.0**-0.05))
    fig.add_vline(x=np.log2(10.0**-0.05))
    if plot_options["title"] != "" or plot_options["title"] is not None:
        plot_title = plot_options["title"] + " " + label
    else:
        plot_title = None
    if not plot_options["xlimits"] or plot_options["xlimits"] == "[]" or \
            not isinstance(plot_options["xlimits"], list):
        xlimits = None
    else:
        xlimits = plot_options["xlimits"]

    if not plot_options["ylimits"] or plot_options["ylimits"] == "[]" or \
            not isinstance(plot_options["ylimits"], list):
        ylimits = None
    else:
        ylimits = plot_options["ylimits"]

    fig.update_layout(
        font=plot_options["font"],

        showlegend=False,
        title=plot_title,
        xaxis=dict(title=dict(
            text=plot_options["X Title"]), range=xlimits),
        yaxis=dict(title=dict(
            text=plot_options["Y Title"]), range=ylimits),
        annotations=total_labels,
        plot_bgcolor='white',
        paper_bgcolor='white',

    )

    
    return fig

In [95]:
# ###PCA plots####
def PCA_plots(data_object, plot_options, saved_settings,username=None):
    """_Prepare data for creating intensity PCA plots (two groups)_
    """
    group_names = []

    # no compare groups is provided, compare first two
    for each_key in plot_options["compare groups"]:
        if each_key in plot_options["compare groups"] and saved_settings["Order@Conditions"]:
            group_names.append(each_key)
    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # this will contain list of run names list for each groups
    #print(saved_settings)
    for eachGroup in group_names:
        runname_sublist = saved_settings[eachGroup]["records"]
        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1       
        #print((runname_sublist))

    all_runs =[item for sublist in runname_list for item in sublist]

    # combined the data after filtering
    #  missing values and log2 transformation/normalization
    combined_infodata = pd.DataFrame() # store run names and group names
    combined_pcaData = pd.DataFrame() # store normalized data and protein names
    for eachGroup in group_names:

        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup])
        normalized_data = NormalizeToMedian(
             current_condition_data["protein_abundance"],apply_log2=True)
        toFileDict = dict(zip(data_object["run_metadata"]["Run Identifier"],data_object["run_metadata"]["Run Names"]))
        toFileDict = generate_column_to_name_mapping(normalized_data.columns, toFileDict)
        normalized_data.rename(columns = toFileDict,inplace=True)

        combined_infodata= pd.concat([combined_infodata, pd.DataFrame({
            "Sample_Groups": normalized_data
            .drop(
                "Accession", axis=1).rename(columns = toFileDict).columns,
            "Type": eachGroup})])
        
        '''for x in range(len(list(combined_infodata["Sample_Groups"]))):
            print(list(combined_infodata["Sample_Groups"])[x])
        '''
        if combined_pcaData.empty:
            combined_pcaData = normalized_data
            print("Empty")
        else:
            combined_pcaData = pd.merge(combined_pcaData, normalized_data)

    #normalize the data
    # using ratio of current group median value divide by the all groups median 
    # to create a scaling factor magicNUm to scale the each group
    quant_names = group_names
    while "Annotated Sequence" in quant_names:
        quant_names.remove("Annotated Sequence")
    while "Accession" in quant_names:
        quant_names.remove("Accession")

    while "Annotated Sequence" in all_runs:
        all_runs.remove("Annotated Sequence")
    while "Accession" in all_runs:
        all_runs.remove("Accession")

    for n in range(len(quant_names)): #-2 because not Accession and Annotated Sequence 
        if "Annotated Sequence" in runname_list[n]:
            runname_list[n].remove("Annotated Sequence")
        if "Accession" in runname_list[n]:
            runname_list[n].remove("Accession")
        
        magicNum =np.nanmedian(combined_pcaData[runname_list[
            n]].dropna(how='all').to_numpy()) /\
                np.nanmedian(combined_pcaData[
            all_runs].dropna(how='all').to_numpy()) 
        for col in combined_pcaData[runname_list[
                n]].columns:
            combined_pcaData[col] = combined_pcaData[col]/magicNum


    #performs k-Nearest Neighbors imputation to fill in any missing values
    combined_pcaData = impute_knn(combined_pcaData)
    combined_infodata.reset_index(drop=True, inplace=True)
    

    # perform PCA transform
    combined_pcaData, exp_var_pca = CalculatePCA(combined_pcaData,
                                                     combined_infodata)

    return plot_PCA_plotly(combined_pcaData,
                           exp_var_pca,
                           plot_options=plot_options,
                           username=username,
                           )


def plot_PCA_plotly(pca_panda,
                    exp_var_pca,
                    plot_options=None,
                    username=None,):

    CSV_link = None
    SVG_link = None

    # Assuming pca_data is a pandas dataframe containing PCA results
    # and "Type" is a column in the dataframe indicating the type of sample
    if not plot_options["xlimits"] or plot_options["xlimits"] == "[]" or \
            not isinstance(plot_options["xlimits"], list):
        xlimits = None
    else:
        xlimits = plot_options["xlimits"]

    if not plot_options["ylimits"] or plot_options["ylimits"] == "[]" or \
            not isinstance(plot_options["ylimits"], list):
        ylimits = None
    else:
        ylimits = plot_options["ylimits"]

    fig = px.scatter(pca_panda,
                     x='PC1',
                     y='PC2',
                     color="Type",
                     text="Sample_Groups",
                     symbol="Type",
                     color_discrete_sequence=plot_options["color"],

                     symbol_sequence=plot_options["symbol"],
                     size_max=30,
                     labels={'PC1': f'PC1 ({round(exp_var_pca[0]*100,2)}%)',
                             'PC2': f'PC2 ({round(exp_var_pca[1]*100,2)}%)',
                             'Type': 'Sample Type'}, title='PCA Plot',
                     width=plot_options["width"],
                     height=plot_options["height"],)

    fig.update_traces(
        mode="markers",
        marker=dict(size=plot_options["marker_size"],),
        hovertemplate="%{text}<br>PC1: %{x} <br>PC2: %{y}")
    fig.update_layout(
        plot_bgcolor="rgba(255, 255, 255, 255)",
        paper_bgcolor="rgba(255, 255, 255, 255)",
        font=plot_options["font"],
        title=plot_options["title"],
        xaxis=dict(linecolor='black',
                   showticklabels=False, mirror=True, range=xlimits),
        yaxis=dict(linecolor='black',
                   showticklabels=False, mirror=True, range=ylimits),
    )
    if WRITE_OUTPUT:
        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_PCA_Plot.svg"), format = "svg", validate = False, engine = "kaleido")
        # create the download CSV and its link
        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        pca_panda.to_csv(os.path.join(
            data_dir, f"{username}_PCA.csv"), index=False)
        print("Downloading links...")
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_PCA.csv"

        # download SVG link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_PCA_Plot.svg"

    return fig, CSV_link, SVG_link

In [96]:
def heatmap_plots(data_object, plot_options, saved_settings, username=None):
    group_names = []

    # no compare groups is provided, compare first two
    for each_key in plot_options["compare groups"]:
        if each_key in plot_options["compare groups"] and saved_settings["Order@Conditions"]:
            group_names.append(each_key)
    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # this will contain list of run names list for each groups
    #print(saved_settings)
    for eachGroup in group_names:
        runname_sublist = saved_settings[eachGroup]["records"]
        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        i += 1       
        #print((runname_sublist))

    all_runs =[]



    # combined the data after filtering
    #  missing values and log2 transformation/normalization
    combined_infodata = pd.DataFrame() # store run names and group names
    combined_heatmap_data = pd.DataFrame() # store normalized data and protein names
    
    for eachGroup in group_names:

        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup])
        normalized_data = NormalizeToMedian(
             current_condition_data["protein_abundance"],apply_log2=False) #apply this later
        toFileDict = dict(zip(data_object["run_metadata"]["Run Identifier"],
                              [eachGroup + "_#" + str(i) for i in range(len(data_object["run_metadata"]["Run Identifier"]))]))
        
        
        toFileDict = generate_column_to_name_mapping(normalized_data.columns, toFileDict)
        normalized_data.rename(columns = toFileDict,inplace=True)
        current_runs = normalized_data.columns.tolist()
        while "Annotated Sequence" in current_runs:
            current_runs.remove("Annotated Sequence")
        while "Accession" in current_runs:
            current_runs.remove("Accession")
        all_runs = all_runs + current_runs
        runname_list.append(current_runs)

        combined_infodata= pd.concat([combined_infodata, pd.DataFrame({"Sample_Groups": normalized_data.drop(
                "Accession", axis=1).rename(columns = toFileDict).columns,
            "Type": eachGroup})])
        
        '''for x in range(len(list(combined_infodata["Sample_Groups"]))):
            print(list(combined_infodata["Sample_Groups"])[x])
        '''
        if combined_heatmap_data.empty:
            combined_heatmap_data = normalized_data
            print("Empty")
        else:
            combined_heatmap_data = pd.merge(combined_heatmap_data, normalized_data)

    #normalize the data
    # using ratio of current group median value divide by the all groups median 
    # to create a scaling factor magicNUm to scale the each group
    quant_names = group_names
    while "Annotated Sequence" in quant_names:
        quant_names.remove("Annotated Sequence")
    while "Accession" in quant_names:
        quant_names.remove("Accession")

    while "Annotated Sequence" in all_runs:
        all_runs.remove("Annotated Sequence")
    while "Accession" in all_runs:
        all_runs.remove("Accession")

    print(all_runs)

    # display(combined_heatmap_data)
        
    print(len(quant_names))
    print(combined_heatmap_data.columns)

    for n in range(len(quant_names)): #-2 because not Accession and Annotated Sequence 
        if "Annotated Sequence" in runname_list[n]:
            runname_list[n].remove("Annotated Sequence")
        if "Accession" in runname_list[n]:
            runname_list[n].remove("Accession")
        
        
        magicNum =np.nanmedian(combined_heatmap_data[runname_list[n]].dropna(how='all').to_numpy()) /\
                np.nanmedian(combined_heatmap_data[
            all_runs].dropna(how='all').to_numpy()) 
        for col in combined_heatmap_data[runname_list[
                n]].columns:
            combined_heatmap_data[col] = combined_heatmap_data[col]/magicNum

    
    if WRITE_OUTPUT or plot_options["significant_only"]:
        

        # How to transpose :(
        # detect differentially expressed proteins
        # reformat dataframe
        new_names = []
        for x in combined_heatmap_data.columns:
            if x.split(sep="_#")[0] in group_names:
                new_names.append("Intensity" + x)
            else:
                new_names.append(x)
        renamed_data = combined_heatmap_data.copy()
        renamed_data.columns = new_names
        # display(combined_heatmap_data)
        long_data = pd.wide_to_long(renamed_data.reset_index(),
                                       stubnames="Intensity",i="Accession",j="Sample",suffix=".*").reset_index()
        
        # if plot_options["agg_groups"]:
        #     # long_data= long_data.groupby(["Accession", "Group"]).agg({"Intensity": "mean"}).reset_index()

        #     # display(long_data)
        #     index = "Group"
        #     pivoted_data =  long_data.pivot(index=index,columns="Accession", values = "Intensity").reset_index()
        # else:
        #     index = "Sample"
        pivoted_data =  long_data.pivot(index="Sample",columns="Accession", values = "Intensity").reset_index()
        pivoted_data["Group"] = pivoted_data["Sample"].str.replace("_#.*","",regex=True)
        # new_cols = combined_heatmap_data.columns.drop("Accession")
        # display(pivoted_data)
        pvalues = []
        means = []
        for col in pivoted_data.columns.drop(["Group","Sample"]):
            if (pivoted_data[col].dropna().shape[0]>=2):
                pvalues.append(anova(*[pivoted_data.loc[pivoted_data["Group"]==x,col].dropna() for x in group_names])[0])
                means.append([np.nanmean(pivoted_data.loc[pivoted_data["Group"]==x,col].dropna()) for x in group_names])
                # print()
            else:
                pvalues.append(np.nan)
                means.append([np.nan])

        fold_changes = []
        # print(means)
        for eachprotein in means:
            length = len(eachprotein)
            total = np.nansum(eachprotein)
            current = []
            if type(eachprotein) != list:
                print(str(eachprotein) + "******************")
                current = [np.nan for x in group_names]
            else:
                for eachMean in eachprotein:
                    each_fold_change = eachMean/((total-eachMean)/(length-1)) #don't include in comparison
                    current.append(each_fold_change)
            fold_changes.append(current)

        significances = []
        alpha = plot_options["alpha"] * 100 # anova function gives p values in %
        log_min_FC = np.log2(plot_options["min_fold_change"])
        for i in range(len(fold_changes)):
            current = False
            current_FCs = fold_changes[i]
            current_p = pvalues[i]
            if type(current_FCs) == list:
                for each_FC in current_FCs:
                    if current_p < alpha and abs(np.log2(each_FC)) > log_min_FC:
                        current = True
            significances.append(current)
        combined_heatmap_data["adjusted_p_values"] = multipletests(pvalues, method='fdr_bh')[1] #adjust 
        combined_heatmap_data["fold_changes"] = fold_changes
        combined_heatmap_data["significant"] = significances



    if WRITE_OUTPUT:
        i = 0
        log_min_FC = np.log2(plot_options["min_fold_change"]) # anova function gives p values in %
        for eachGroup in group_names:
            if eachGroup not in combined_heatmap_data.columns:
                j = 0
                current = []
                for x in combined_heatmap_data["fold_changes"]:
                    current.append(combined_heatmap_data["significant"][j] and abs(np.log2(x[i]))>log_min_FC)
                    j = j + 1
                combined_heatmap_data[eachGroup] = current
            else:
                print("ERROR: don't use group names that are also accession numbers")
            i = i + 1

        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        combined_heatmap_data.to_csv(os.path.join(
            data_dir, f"{username}_Heatmap.tsv"),sep="\t", index=False)
        
    if plot_options["significant_only"]:
        print(combined_heatmap_data.shape)
        combined_heatmap_data = combined_heatmap_data[combined_heatmap_data["significant"]]
        print(combined_heatmap_data.shape)
        combined_heatmap_data = combined_heatmap_data.drop(columns=["significant","adjusted_p_values","fold_changes"]+group_names)
    elif WRITE_OUTPUT:
        combined_heatmap_data = combined_heatmap_data.drop(columns=["significant","adjusted_p_values","fold_changes"]+group_names)

    if plot_options["log2_transform"]:
        for eachCol in combined_heatmap_data.columns.drop("Accession"):
            combined_heatmap_data[eachCol] = np.log2(combined_heatmap_data[eachCol])

    figure = px.imshow(combined_heatmap_data.set_index("Accession").dropna(), aspect="auto")
    CSV_link = None
    SVG_link = None
    return figure, CSV_link, SVG_link 

In [97]:
####All the functions and constants imports above############################
### All the following sections are for configuration and plotting############

In [98]:
"""_This section read in data from the data management system or from the input
files.
Method 1 use process queue ID, serve address, account and password to read
from data manage process queue_
How to use: set process_queue_id and server_address, username and password
"""

process_queue_id =7272 # example MM PD3 6983, FP 7272
server_address = "10.37.240.41"
username = "XiaofengXie" #user name
password = "" # DO NOT LEAVE your password when uploading to github
if password != "" or process_queue_id == None:
    queue_info, processor_info = queue_info_api(process_queue_id,
                                                    server_address,
                                                    username,
                                                    password)
    data_obj = read_file(queue_info=queue_info, processor_info=processor_info)
else:
    queue_info, processor_info = None, None


In [99]:
#Method put files in, one dictionary for each analysis, if there is only one file or you are looking in all files, then you don't need @'s in your SETTINGS_FILE
filelist = [
             {"input1":"20min_January/report.pg_matrix.tsv",
             "input2":"20min_January/report.pr_matrix.tsv",
             "input3":"20min_January/report-first-pass.pg_matrix.tsv",
             "input4":"20min_January/report-first-pass.pr_matrix.tsv",
             "input5":"Input/Payne2/Payne_organoids2.0_InputFiles.txt",
             "process_app": "DIANN"},
          
            ]

            #Order of files
                #For FragPipe
                    #input1= Protein with MBR
                    #input2= Peptide with MBR
                #For DIANN
                    #input1= pg_matrix
                    #input2= pr_matrix 
                    #input3= pg_matrix MS2
                    #input4= pr_matrix MS2
                #For PD
                    #input1= Protein 
                    #input2= Peptide Groups
                    #input3= blank
                    #input4= blank
                    #input5= input_files


SETTINGS_FILE = "settings_test.txt" #tab delimited
    #Format
    #Required Columns
        #Group Name	
        #filter_in: a string pattern contained in desired raw filenames in analysis followed by an @ + the file number (filelist index) to look in
        #filter_out: a string pattern contained in undesired raw filenames
    #Extra columns
        #can be used to group by color, layer, or x position in CV and ID graphs

In [100]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
x = read_files(grouped_input_files=filelist)
data_obj = outer_join_data_objects(x)

settings_table = pd.read_table(SETTINGS_FILE,sep="\t")
saved_settings = settings_table.set_index("Conditions").to_dict(orient="index")
#any run with any of the filter_out items will not be used.

for eachGroup in saved_settings:
    i = 0
    saved_settings[eachGroup]["records"] = []
    filterOutType = type(saved_settings[eachGroup]["filter_out"])
    if filterOutType == str or filterOutType == int or filterOutType == float and not pd.isna(saved_settings[eachGroup]["filter_out"]):
        filterOut = str.split(saved_settings[eachGroup]["filter_out"],sep = ",")
    else:
        filterOut = ["M@di"]
    if len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) > 1: #multiple files, only some have the runs for this group
        user_list = []
        #add all runs from all analyses to be probed
        for each_fileID in str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[1:]:
            for eachIdentifier in data_obj["run_metadata"]["Run Identifier"]:    
                currentRun = data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == eachIdentifier]["Run Names"] 
                if currentRun.size == 1:
                    if each_fileID == str.split(eachIdentifier,sep="-")[0] and list(currentRun)[0] not in user_list:
                        user_list.append(list(currentRun)[0])
                else:
                    pass
        #filter for runs that relate to this gorup within those analyses
        for run_name in user_list:
            if str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[0] in run_name and (not any(item in run_name for item in filterOut)):
                saved_settings[eachGroup]["records"].append(list(data_obj["run_metadata"][data_obj["run_metadata"]["Run Names"] == run_name]["Run Names"])[0]) 
            else:
                pass
    elif len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) == 1: #across all files or maybe there is only one
            for run_name in data_obj["run_metadata"]["Run Names"]:
                if str(saved_settings[eachGroup]["filter_in"]) in run_name and (not any(item in run_name for item in filterOut)):
                    saved_settings[eachGroup]["records"].append(data_obj["run_metadata"]["Run Names"][i])  
                i = i + 1   

#add the order of each column
ignore_columns = ["filter_in","filter_out"]
category_columns = [x for x in settings_table.columns.to_list() if x not in ignore_columns]

for eachCol in category_columns:
    saved_settings["Order@"+eachCol] = settings_table[eachCol].drop_duplicates().to_list()
    

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\319232975.py:25: UserWarning:

Boolean Series key will be reindexed to match Dat

***


In [101]:
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "Y Title": "Protein Identification (MBR)",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "total",#grouped, total, MS2, grouped_stacked, stacked
            "Group By X": "Sample Amount", #ID_Mode does MS2 vs MBR
            "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
            "Group By Stack": "ID_Mode",#ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide

# display(data_obj["peptide_ID_Summary"])
# pd.merge(data_obj["peptide_ID_Summary"],data_obj["run_metadata"]).to_csv("csv/pep.csv")
# display(data_obj["run_metadata"])
# display(data_obj["protein_ID_Summary"])
# pd.merge(data_obj["protein_ID_Summary"],data_obj["run_metadata"]).to_csv("csv/pep.csv")

figure ,_ ,_ =ID_plots(data_obj, plot_options, saved_settings,username="20min")
figure.update_layout(showlegend=False,xaxis_title=None)
figure.write_image("20min_IDs.png", scale=10, width = 1000, height=1000)
figure.show()


In [102]:
# CV violin plot
plot_options={    
        "median label": "True", #only works for total and MS2, can be found in CV_summary if you set WRITE_OUTPUT = True
        "box": False,
        "X Title": "Conditions (protein)",
        "Y Title": "CV of Abundance (%)",
        "color": ["#3E6990", "#7aa8e6","#C6878F", "#fac8d3","#AABD8C", "#E9E3B4",  "#F39B6D", "#C6878F", "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "ylimits": [-10, 400],
        "CV mode": "MS2",#grouped, total, MS2, grouped_stacked, stacked
        "Group By X": "Gradient Length", #ID_Mode does MS2 vs MBR
        "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
        "Group By Stack": "ID_Mode" #ID_Mode does MS2 vs MBR
    }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =CV_plots(data_obj, plot_options, saved_settings,username="20min")
figure.show()


C:\Users\keiwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\nanfunctions.py:1216: RuntimeWarning:

Mean of empty slice



MS2 CVs...


In [103]:
# ID Venns plot
plot_options={
            "compare groups": ["Colony 1.3","Colony 1.6"],
            "title": "Venn Diagram (peptide)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =venns_plots(data_obj, plot_options, saved_settings,missing_values_max=33,username="20min")
figure.show()



In [104]:
# Protein Abundance Volcano plot
plot_options={
            "compare groups": ["Colony 1.3","Colony 1.6"],
            "title": "Volcano Plot",
            "X Title": "Log10 Fold Change",
            "Y Title": "-Log10(adjusted p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =volcano_plots(data_obj, plot_options, saved_settings,missing_values_max=33,username="20min_funky")
figure.show()


In [105]:
# Protein Abundance PCA plot
plot_options={
            "compare groups": ["Colony 1.3","Colony 1.6"],
            "title": "PCA Analysis",
            "color": ["#AABD8C", "#ccc066","blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 18,
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =PCA_plots(data_obj, plot_options, saved_settings,username="20min")
figure.update_layout(title=None,showlegend=False)
figure.write_image("20min_PCA.png", scale=10, width = 1000, height=1000)
figure.show()


Empty


In [106]:
# Protein Abundance Heat Map
plot_options={
            "compare groups":  ["Colony 1.3","Colony 1.6","Bulk 200pg"],
            "log2_transform": False,
            "significant_only": True,
            "alpha": 0.05, 
            "min_fold_change": 2
        }
figure, _, _ =heatmap_plots(data_obj, plot_options, saved_settings,username="20min")
figure.show()

Empty
['Colony 1.3_#0', 'Colony 1.3_#3', 'Colony 1.3_#5', 'Colony 1.3_#11', 'Colony 1.3_#13', 'Colony 1.3_#15', 'Colony 1.3_#16', 'Colony 1.3_#18', 'Colony 1.3_#21', 'Colony 1.3_#22', 'Colony 1.3_#25', 'Colony 1.3_#27', 'Colony 1.3_#30', 'Colony 1.3_#33', 'Colony 1.3_#35', 'Colony 1.3_#37', 'Colony 1.3_#41', 'Colony 1.3_#42', 'Colony 1.3_#46', 'Colony 1.3_#48', 'Colony 1.3_#50', 'Colony 1.6_#1', 'Colony 1.6_#2', 'Colony 1.6_#4', 'Colony 1.6_#7', 'Colony 1.6_#9', 'Colony 1.6_#12', 'Colony 1.6_#14', 'Colony 1.6_#17', 'Colony 1.6_#20', 'Colony 1.6_#23', 'Colony 1.6_#26', 'Colony 1.6_#28', 'Colony 1.6_#29', 'Colony 1.6_#32', 'Colony 1.6_#34', 'Colony 1.6_#36', 'Colony 1.6_#40', 'Colony 1.6_#44', 'Colony 1.6_#45', 'Colony 1.6_#49', 'Colony 1.6_#51', 'Bulk 200pg_#38', 'Bulk 200pg_#62', 'Bulk 200pg_#63', 'Bulk 200pg_#64', 'Bulk 200pg_#65', 'Bulk 200pg_#66', 'Bulk 200pg_#67', 'Bulk 200pg_#68']
3
Index(['Accession', 'Colony 1.3_#0', 'Colony 1.3_#3', 'Colony 1.3_#5',
       'Colony 1.3_#11', 'Co

C:\Users\keiwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning:

at least one input has length 0

C:\Users\keiwe\AppData\Local\Temp\ipykernel_10708\1244416272.py:136: RuntimeWarning:

Mean of empty slice



(1713, 57)
(10, 57)
